In [625]:
import pandas as pd
from datetime import datetime, timedelta
from dateutil import parser
import re
from unidecode import unidecode
file_path = "Opportunity Data.xlsx"  
df = pd.read_excel(file_path)
print(df.head())
df.shape

  Learner SignUp DateTime                  Opportunity Id  \
0     06/14/2023 12:30:35  00000000-0GN2-A0AY-7XK8-C5FZPP   
1     2023-01-05 05:29:16  00000000-0GN2-A0AY-7XK8-C5FZPP   
2     2023-09-04 20:35:08  00000000-0GN2-A0AY-7XK8-C5FZPP   
3     08/29/2023 05:20:03  00000000-0GN2-A0AY-7XK8-C5FZPP   
4     2023-06-01 15:26:36  00000000-0GN2-A0AY-7XK8-C5FZPP   

                                    Opportunity Name Opportunity Category  \
0  Career Essentials: Getting Started with Your P...               Course   
1  Career Essentials: Getting Started with Your P...               Course   
2  Career Essentials: Getting Started with Your P...               Course   
3  Career Essentials: Getting Started with Your P...               Course   
4  Career Essentials: Getting Started with Your P...               Course   

  Opportunity End Date        First Name        Date of Birth  Gender  \
0  06/29/2024 18:52:39             Faria  2001-12-01 00:00:00  Female   
1  06/29/2024 18:52:39  

(8558, 16)

In [574]:
df.shape

(8558, 16)

In [507]:
print(df.dtypes)

Learner SignUp DateTime            object
Opportunity Id                     object
Opportunity Name                   object
Opportunity Category               object
Opportunity End Date               object
First Name                         object
Date of Birth                      object
Gender                             object
Country                            object
Institution Name                   object
Current/Intended Major             object
Entry created at           datetime64[ns]
Status Description                 object
Status Code                         int64
Apply Date                         object
Opportunity Start Date             object
dtype: object


In [509]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8558 entries, 0 to 8557
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Learner SignUp DateTime  8558 non-null   object        
 1   Opportunity Id           8558 non-null   object        
 2   Opportunity Name         8558 non-null   object        
 3   Opportunity Category     8558 non-null   object        
 4   Opportunity End Date     8558 non-null   object        
 5   First Name               8558 non-null   object        
 6   Date of Birth            8558 non-null   object        
 7   Gender                   8558 non-null   object        
 8   Country                  8558 non-null   object        
 9   Institution Name         8553 non-null   object        
 10  Current/Intended Major   8553 non-null   object        
 11  Entry created at         8558 non-null   datetime64[ns]
 12  Status Description       8558 non-

In [627]:
def clean_date_columns(df, date_columns=None):
    if date_columns is None:
        date_columns = [
            'Learner SignUp DateTime',
            'Opportunity End Date',
            'Entry created at',
            'Apply Date',
            'Opportunity Start Date'
        ]
    
    EXCEL_BASE_DATE = datetime(1899, 12, 30)
    
    def parse_date_flexible(value):
        if pd.isna(value):
            return pd.NaT
    
        if isinstance(value, datetime):
            return value
        
        value = str(value).strip()

        if 'days' in value:
            try:
                days = float(re.search(r'(\d+)', value).group(1))
                return EXCEL_BASE_DATE + timedelta(days=days)
            except:
                return pd.NaT
        
        value = re.sub(r'24:(\d{2}:\d{2})', r'23:\1', value)
        
        try:
            dt = parser.parse(value, fuzzy=True, dayfirst=False, yearfirst=False)
            return dt
        except:
            return pd.NaT
    
    for col in date_columns:
        if col in df.columns:
            df[col] = df[col].apply(parse_date_flexible)
            # uniform string format
            df[col] = df[col].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S') if pd.notna(x) else pd.NaT)
            # Convert back to datetime type
            df[col] = pd.to_datetime(df[col], errors='coerce')
    
    return df
df = clean_date_columns(df)
print(df.head(5))

  Learner SignUp DateTime                  Opportunity Id  \
0     2023-06-14 12:30:35  00000000-0GN2-A0AY-7XK8-C5FZPP   
1     2023-01-05 05:29:16  00000000-0GN2-A0AY-7XK8-C5FZPP   
2     2023-09-04 20:35:08  00000000-0GN2-A0AY-7XK8-C5FZPP   
3     2023-08-29 05:20:03  00000000-0GN2-A0AY-7XK8-C5FZPP   
4     2023-06-01 15:26:36  00000000-0GN2-A0AY-7XK8-C5FZPP   

                                    Opportunity Name Opportunity Category  \
0  Career Essentials: Getting Started with Your P...               Course   
1  Career Essentials: Getting Started with Your P...               Course   
2  Career Essentials: Getting Started with Your P...               Course   
3  Career Essentials: Getting Started with Your P...               Course   
4  Career Essentials: Getting Started with Your P...               Course   

  Opportunity End Date        First Name        Date of Birth  Gender  \
0  2024-06-29 18:52:39             Faria  2001-12-01 00:00:00  Female   
1  2024-06-29 18:52:39  

In [605]:
df['Date of Birth'] = pd.to_datetime(df['Date of Birth'], errors='coerce')
df['Date of Birth'] = df['Date of Birth'].dt.date

string_columns = ["First Name", "Country", "Institution Name", "Current/Intended Major", "Opportunity Name", "Opportunity Category", "Status Description", "Gender"]
df[string_columns] = df[string_columns].astype("string")

print(df.dtypes)

Learner SignUp DateTime    datetime64[ns]
Opportunity Id                     object
Opportunity Name           string[python]
Opportunity Category       string[python]
Opportunity End Date       datetime64[ns]
First Name                 string[python]
Date of Birth                      object
Gender                     string[python]
Country                    string[python]
Institution Name           string[python]
Current/Intended Major     string[python]
Entry created at           datetime64[ns]
Status Description         string[python]
Status Code                         int64
Apply Date                 datetime64[ns]
Opportunity Start Date     datetime64[ns]
dtype: object


In [629]:
columns_to_clean = ['Country', 'Institution Name', 'Current/Intended Major', 'First Name']

def clean_text(text):
    if pd.isna(text):
        return pd.NA  
    text = str(text)
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)       # normalize spaces
    text = re.sub(r'[.]+$', '', text)  # remove trailing . 
    text = text.title()                    # convert to Title Case
    return text

for col in columns_to_clean:
    df[col] = df[col].apply(clean_text)

print(df[columns_to_clean].head())

         Country                  Institution Name Current/Intended Major  \
0       Pakistan                             Nwihs              Radiology   
1          India                       Saint Louis    Information Systems   
2  United States  Illinois Institute Of Technology       Computer Science   
3  United States            Saint Louis University    Information Systems   
4  United States            Saint Louis University       Computer Science   

         First Name  
0             Faria  
1          Poojitha  
2          Emmanuel  
3  Amrutha Varshini  
4    Vinay Varshith  


In [631]:
print(df.info())
print(df.isna().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8558 entries, 0 to 8557
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Learner SignUp DateTime  8558 non-null   datetime64[ns]
 1   Opportunity Id           8558 non-null   object        
 2   Opportunity Name         8558 non-null   object        
 3   Opportunity Category     8558 non-null   object        
 4   Opportunity End Date     8558 non-null   datetime64[ns]
 5   First Name               8558 non-null   object        
 6   Date of Birth            8558 non-null   object        
 7   Gender                   8558 non-null   object        
 8   Country                  8558 non-null   object        
 9   Institution Name         8553 non-null   object        
 10  Current/Intended Major   8553 non-null   object        
 11  Entry created at         8558 non-null   datetime64[ns]
 12  Status Description       8558 non-

In [633]:
# Compute mode per group and broadcast with transform
modes = df.groupby('Opportunity Name')['Opportunity Start Date'] \
          .transform(lambda x: x.mode()[0] if not x.mode().empty else pd.NaT)

# Fill missing with group mode
df['Opportunity Start Date'] = df['Opportunity Start Date'].fillna(modes)

#print(df[['Opportunity Name', 'Opportunity Start Date']].sort_values('Opportunity Start Date').to_string(index=False))

In [635]:
distinct_counts = df[['Country', 'Institution Name', 'Current/Intended Major','First Name']].nunique()
print(distinct_counts)

Country                     71
Institution Name          1795
Current/Intended Major     392
First Name                2899
dtype: int64


In [637]:
# Create mapping to make all entities in standard format
country_mapping = {
    'Iran Islamic Republic Of Persian Gulf': 'Iran',
    'Iran, Islamic Republic Of Persian Gulf': 'Iran',
    'Korea, Republic Of South Korea': 'South Korea',
    'Tanzania, United Republic Of Tanzania': 'Tanzania',
    'Libyan Arab Jamahiriya': 'Libya',
    'Virgin Islands, U.S.': 'U.S. Virgin Islands',
    'Falkland Islands (Malvinas)': 'Falkland Islands'
}

df['Country'] = df['Country'].replace(country_mapping)
df['Country'] = df['Country'].str.strip().str.title()

unique_countries = sorted(df['Country'].unique())
print(unique_countries)

['Afghanistan', 'Algeria', 'American Samoa', 'Australia', 'Azerbaijan', 'Bangladesh', 'Belarus', 'Bhutan', 'Botswana', 'Brazil', 'British Indian Ocean Territory', 'Cameroon', 'Canada', 'China', 'Congo', "Cote D'Ivoire", 'Dominican Republic', 'Egypt', 'Ethiopia', 'Falkland Islands', 'France', 'Gambia', 'Germany', 'Ghana', 'Honduras', 'India', 'Indonesia', 'Iran', 'Iraq', 'Ireland', 'Japan', 'Kazakhstan', 'Kenya', 'Lebanon', 'Lesotho', 'Liberia', 'Libya', 'Malaysia', 'Mauritius', 'Morocco', 'Namibia', 'Nepal', 'Nigeria', 'Pakistan', 'Peru', 'Philippines', 'Qatar', 'Rwanda', 'Saudi Arabia', 'Sierra Leone', 'Singapore', 'Somalia', 'South Africa', 'South Korea', 'Spain', 'Sri Lanka', 'Tanzania', 'Tunisia', 'Turkey', 'Uganda', 'Ukraine', 'United Arab Emirates', 'United Kingdom', 'United States', 'Uzbekistan', 'Vietnam', 'Virgin Islands, U.S', 'Yemen', 'Zambia', 'Zimbabwe']


In [639]:
#corrections and invalid names
NAME_CORRECTIONS = {
    "zynb n`mn": "Zaynab Numan",
    "grimaa": "Garima"
}
INVALID_NAMES = ["M", "C", "S", "Sk", "Sp", "Md", "Fnu", "Rb", "G.", 
                 "E", "V", "K", "I'm", "G", "Nu", "Om", "Mr", "De", "F"]
def clean_first_name(name):
    if pd.isna(name):
        return pd.NA 
    # Convert to string, strip spaces
    name = str(name).strip()
    # Transliterate to ASCII
    name_ascii = unidecode(name).lower()
    # Apply corrections first
    if name_ascii in NAME_CORRECTIONS:
        return NAME_CORRECTIONS[name_ascii]
    # Remove names that are only digits, dashes, or spaces
    if re.fullmatch(r'[\d\-\s]+', name_ascii):
        return pd.NA  
    # Remove words with digits
    words = [w for w in name_ascii.split() if not any(c.isdigit() for c in w)] 
    # Remove invalid words
    words = [w for w in words if w.title() not in INVALID_NAMES]    
    if not words:
        return pd.NA
    cleaned_name = ' '.join([w.capitalize() for w in words])
    # Discard initials like "A." or "B"
    if re.fullmatch(r"[A-Za-z]\.?", cleaned_name):
        return pd.NA
    # Minimum length check
    if len(cleaned_name) < 3:
        return pd.NA 
    return cleaned_name
# Apply to your DataFrame
df['First Name'] = df['First Name'].apply(clean_first_name)

# Preview
print(df['First Name'].head(5))

0               Faria
1            Poojitha
2            Emmanuel
3    Amrutha Varshini
4      Vinay Varshith
Name: First Name, dtype: object


In [641]:
# Create mapping to make all entities in standard format and remove invalid entities
import pandas as pd
from unidecode import unidecode
mapping = {
    'Accounting And Finance': 'Accounting',
    'Accountancy': 'Accounting',
    'Data Analysis': 'Data Analytics',
    'Data Analyst': 'Data Analytics',
    'Bsc In Computer Science': 'Computer Science',
    'Computer Science And Engineering': 'Computer Science',
    'Already Graduation Completed': 'Graduated',
    'Bachelor Degree In Nursing': 'BSc Nursing',
    'Working As Software Devloper': 'Software Developer',
    'Cybersecurity': 'Cyber Security',
    'Ms In Supply Chain Management': 'MS Supply Chain Management',
    'Esports And Game Development': 'Sports And Game Development',
    'Pre Engg': 'Pre Engineering',
    'After Bachelor': 'Bachelor',
    'Master In Vlsi And Microelectronics': 'Master In VLSI And Microelectronics',
    'Social S': 'Social Science',
    'Bnsc Nursing Science': 'Bachelor of Nursing Science',
    'Ms In Cybersecurity': 'MS Cyber Security',
    'Monitoring And Evaluation': 'Monitoring And Evaluation',
    'Ms Biostatistics And Health Analytics': 'MS Biostatistics And Health Analytics',
    'Ms In Analytics': 'MS In Analytics',
    'Ms In Information Science Machine Learning': 'MS In Information Science Machine Learning',
    'Msc Cyber Forensics And Security': 'MSc Cyber Forensics And Security',
    'Msc In Computational Decision Science And Operations Research': 'MSc In Computational Decision Science And Operations Research',
    'Msc In Disaster Management And Climate Sustainability Studies': 'MSc In Disaster Management And Climate Sustainability Studies',
    'Msc In Information Technology': 'MSc In Information Technology',
    'Msters In Healthcare Administration': 'Masters In Healthcare Administration',
    'Ms Cyber Security': 'MS Cyber Security',
    'Textile': 'Textile Engineering',
    'Statistics And Computeri Science': 'Statistics And Computer Science',
    'Radiography': 'Radiology',
    'Pursuing Mca': 'Master of Computer Applications',
    'Nurse': 'Nursing',
    'Medicine': 'Medical Sciences',
    'Medical Informatics': 'Medical Sciences',
    'Mechanical': 'Mechanical Engineering',
    'Electrical Engineer': 'Electrical Engineering',
    'Computer And Information Sciencess': 'Computer And Information Sciences',
    'Computer': 'Computer Science',
    'Business Chinese': 'Business Education',
    "Baf": "Bachelor of Accounting and Finance",
    "Bba": "Bachelor of Business Administration",
    "Bba Hr": "Bachelor of Business Administration in Human Resources",
    "Bca": "Bachelor of Computer Applications",
    "Bcom": "Bachelor of Commerce",
    "Bcom International Business": "Bachelor of Commerce in International Business",
    "Bs Forestry And Wildlife Management": "Bachelor of Science in Forestry and Wildlife Management",
    "Bsc": "Bachelor of Science",
    "Bsc Agriculture": "Bachelor of Science in Agriculture",
    "Bsc Computer Science": "Bachelor of Science in Computer Science",
    "Bsc Nursing": "Bachelor of Science in Nursing",
    "Bsc Physician Assistant": "Bachelor of Science in Physician Assistant",
    "Bsit": "Bachelor of Science in Information Technology",
    "Btech": "Bachelor of Technology",
    "Ise": "Information Science and Engineering",
    "It": "Information Technology",
    "Mba": "Master of Business Administration",
    "Mbbs": "Bachelor of Medicine, Bachelor of Surgery",
    "Mca": "Master of Computer Applications",
    "Mmg": "Master of Management",
    "Ms Biostatistics And Health Analytics": "Master of Science in Biostatistics and Health Analytics",
    "Ms Cyber Security": "Master of Science in Cyber Security",
    "Ms In Analytics": "Master of Science in Analytics",
    "Ms In Information Science Machine Learning": "Master of Science in Information Science and Machine Learning",
    "Ms Supply Chain Management": "Master of Science in Supply Chain Management",
    "Msc Cyber Forensics And Security": "Master of Science in Cyber Forensics and Security",
    "Msc In Computational Decision Science And Operations Research": "Master of Science in Computational Decision Science and Operations Research",
    "Msc In Disaster Management And Climate Sustainability Studies": "Master of Science in Disaster Management and Climate Sustainability Studies",
    "Msc In Information Technology": "Master of Science in Information Technology",
    "Ba Psych": "Bachelor of Arts in Psychology",
    "TE": "Telecommunication Engineering",
    "OT": "Occupational Therapy",
    "Te": "Telecommunication Engineering",
    "Ot": "Occupational Therapy",
    "Cycw":"Child and Youth Care"

}
invalid_majors = [
    'Na', 'Nil', 'Non', 'I Am Major', 'Otheraassss', 'Sdada', 'Www', 'Yoganand Sir',
    'Eee', 'Pol', 'Faizan', 'Not Know Ab', 'Ghj', 'Job', 'Fresher', 'Degree', 'Shaista',
    'Other', 'Othe','Oth',  'No', 'Aa', 'Zack', 'Goodcreativity', 
    'Lower Credit', 'Ha', 'To Study', 'Unemployed',  'Pv' , 'Student'
]

def clean_major(major):
    if pd.isna(major):
        return pd.NA
    major = ' '.join(str(major).strip().split())

    major = unidecode(major)

    if major in invalid_majors:
        return pd.NA

    if major in mapping:
        major = mapping[major]

    major = ' '.join([w.capitalize() for w in major.split()])

    return major

df['Current/Intended Major'] = df['Current/Intended Major'].apply(clean_major)
print(df['Current/Intended Major'].unique())


['Radiology' 'Information Systems' 'Computer Science'
 'Mechanical Engineering' 'Artificial Intelligence'
 'Robotics And Automation Engineering' 'Data Visualization'
 'Business Administration' 'Public Health' 'Architecture'
 'Computer Science And Information Systems' 'Biology' 'Economics' <NA>
 'Mathematics' 'Bioinformatics' 'Biomedical Engineering'
 'Electrical And Electronic Engineering' 'Business And Management Studies'
 'Electrical And Computer Engineering' 'Accounting' 'Secretarial'
 'Data Science' 'Statistics' 'Electronics And Communication'
 'Computer Information Systems' 'Management Information Systems'
 'Project Management' 'Medical Sciences' 'Information'
 'Information Technology' 'Actuarial Mathematics' 'Software Engineering'
 'Biological Sciences' 'Urban And Housing Development' 'Human Resources'
 'Cyber Security' 'Data Analytics' 'Computer Engineering'
 'Environmental Sciences' 'Philosophy' 'Law And Legal Studies'
 'Industrial Engineering' 'Theology Or Divinity And Religio

In [643]:
INSTITUTE_CORRECTIONS = {
    "Yan Zhou Shi Shi Yan Wai Yu Xue Xiao": "Yanzhou Experimental Foreign Language School",
    "Zhu Hai Shi Yi Fu Shu Shi Yan Xue Xiao": "Zhuhai Affiliated Experimental School",
    "Zhu Hai Yi Fu Guo Ji Bu": "Zhuhai Affiliated Experimental School",
    "Thnwy@ Bn Syn Lt'Hyly@": "Ibn Sina Secondary School"
}

def clean_institution(name: str) -> str:
    if pd.isna(name):
        return name   # keep NaN as is
    
    # Step 1: Trim spaces
    cleaned = name.strip()
    
    # Step 2: Normalize unicode 
    cleaned = unidecode(cleaned)
    
    # Step 3: Remove unwanted characters like @, multiple spaces
    cleaned = re.sub(r"[@']", "", cleaned)
    cleaned = re.sub(r"\s+", " ", cleaned).strip()
    
    # Step 4: Apply dictionary corrections if available
    if cleaned in INSTITUTE_CORRECTIONS:
        return INSTITUTE_CORRECTIONS[cleaned]
    
    return cleaned

# Apply cleaning
df['Institution Name'] = df['Institution Name'].apply(clean_institution)

In [645]:
# Create mapping to make all entities in standard format and remove invalid entities
INSTITUTE_CORRECTIONS = {
    "Abdul Wali Khan University": "Abdul Wali Khan University",
    "Abdul Wali Khan University Mardan": "Abdul Wali Khan University",
    "Abc Inter College": "ABC Inter College",
    "Abia State University": "Abia State University",
    "Abia State University Uturu.": "Abia State University",
    "Abia State University Uturu ": "Abia State University",
    "Adekunle Ajasin University": "Adekunle Ajasin University",
    "Adekunle Ajasin University Akungba Akoko": "Adekunle Ajasin University",
    "Adekunle Ajasin University Akungba Ondo Nigeria": "Adekunle Ajasin University",
    "Addis Ababa Institute Of Technology": "Addis Ababa Science and Technology University",
    "Addis Ababa Science And Technology University": "Addis Ababa Science and Technology University",
    "Addis Ababa University": "Addis Ababa Science and Technology University",
    "Adeleke University": "Adeleke Ajasin University",
    "African School Of Management And Technologie": "African School Of Management And Technology",
    "African School Of Management Of Technology": "African School Of Management And Technology",
    "Afsm": "African School Of Management And Technology",
    "Agriculture University": "Agriculture University Peshawar",
    "Agriculture University Peshawar": "Agriculture University Peshawar",
    "Ahmadu Bello University": "Ahmadu Bello University Zaria",
    "Ahmadu Bello University Zaria": "Ahmadu Bello University Zaria",
    "Ahmadu Bello University Zaria Kaduna State": "Ahmadu Bello University Zaria",
    "Ahmadu Bello University Zaria Kaduna State Nigeria": "Ahmadu Bello University Zaria",
    "Ahmadu Bello University Zaria Nigeria.": "Ahmadu Bello University Zaria",
    "Allama Iqbal Open Universty": "Allama Iqbal Open University",
    "Allama Iqbal University": "Allama Iqbal Open University",
    "Al-Nahrain": "Al-Nahrain University",
    "Al-Nahrain University": "Al-Nahrain University",
    "Alva'S Institute Of Engineering And Technology": "Alva's Institute Of Engineering And Technology",
    "Ambalika Institue Of Management And Technology": "Ambalika Institute Of Management And Technology",
    "Ambalika Institute Of Management And Technology": "Ambalika Institute Of Management And Technology",
    "Ambrose Alli University": "Ambrose Alli University",
    "Ambrose Alli University Ekpoma": "Ambrose Alli University",
    "Amity International School Noida": "Amity International School Noida",
    "Amity University": "Amity University",
    "Amity University Online": "Amity University",
    "Amity University Patna": "Amity University",
    "Amity Unniversity Punjab": "Amity University",
    "Amrita Vishwa Vidyapeetham": "Amrita Vishwa Vidyapeetham",
    "Amrita Vishwa Vidyapeetham Kerala": "Amrita Vishwa Vidyapeetham",
    "Amrita Vishwa Vishyapeetham": "Amrita Vishwa Vidyapeetham",
    "Anantha Lakshmi Institute Of Technology And Sciences": "Ananthalakshmi Institute Of Technology And Sciences",
    "Ananthalakshmi Institute Of Technology And Sciences": "Ananthalakshmi Institute Of Technology And Sciences",
    "Avanthi'S Scientific Technological And Research Academy": "Avanthi's Scientific Technological And Research Academy",
    "Ashoka": "Ashoka Academy",
    "Ashoka Academy": "Ashoka Academy",
    "Bahria University": "Bahria University",
    "Bahria University Karachi Campus": "Bahria University",
    "Bldea'S V P Dr Pg Halakatti College Of Engineering & Technology College In Bijapur Karnataka": "Bldea's V P Dr Pg Halakatti College Of Engineering & Technology College In Bijapur Karnataka",
    "Cmr": "Cmr Instutie Of Technology",
    "Cmr Institute Of Technology Hyderabad": "Cmr Institute Of Technology",
    "Colonel'S Central Academy": "Colonel's Central Academy",
    "Comsats": "Comsats University Islamabad",
    "Comsats University Islamabad": "Comsats University Islamabad",
    "Comsats University Islamabad Attock Campus": "Comsats University Islamabad",
    "Comsats University Islamabad Campus": "Comsats University Islamabad",
    "Day Science College Kano.": "Day Science College Kano",
    "Deen Dayal Upadhyaya College University Of Delhi": "Deen Dayal Upadhyaya College, University of Delhi",
    "Deen Dayal Upadhyaya Gorakhpur University": "Deen Dayal Upadhyaya Gorakhpur University",
    "Delhi Public School": "Delhi Public School",
    "Delhi Public School Bangalore East": "Delhi Public School",
    "Delhi Public School R.K. Puram": "Delhi Public School",
    "Delta State University": "Delta State University",
    "Delta State University Abraka": "Delta State University",
    "Dhanalakshmi Srinivasan Engineering College": "Dhanalakshmi Srinivasan Engineering College",
    "Dhanalakshmi Srinivasan Engineering College Perambalur": "Dhanalakshmi Srinivasan Engineering College",
    "Dhanalakshmi Srinivasan Engineering Collge": "Dhanalakshmi Srinivasan Engineering College",
    "Dps Dwarka": "DPS",
    "Dps Ruby Park": "DPS",
    "Dr D Y Patil College Of Engineering And Innovation Pune": "Dr D Y Patil College Of Engineering And Innovation",
    "Dr D.Y. Patil College Of Engineering And Innovation": "Dr D Y Patil College Of Engineering And Innovation",
    "Dr Dy Patil College Of Engineering And Innovation": "Dr D Y Patil College Of Engineering And Innovation",
    "Dr Dy Patil School Of Science And Technology": "Dr D Y Patil School Of Science And Technology",
    "Dr.D Y Patil Schhol Of Science And Technology": "Dr D Y Patil School Of Science And Technology",
    "Dr.D.Y.Patil School Of Science And Technology Pune": "Dr D Y Patil School Of Science And Technology",
    "Dseu": "DSEU",
    "Du": "DSEU",
    "Dvr &Dr.Hs Mic College Of Technology": "DVR & Dr. HS MIC College Of Technology",
    "Dwarkadas J Sanghvi College Of Engineering": "Dwarkadas J. Sanghvi College Of Engineering",
    "Dwarkadas J. Sanghvi College Of Engineering": "Dwarkadas J. Sanghvi College Of Engineering",
    "Federal Science And Technical College Uromi.": "Federal Science And Technical College Uromi",
    "Egerton University": "Egerton University",
    "Egerton University Niversity": "Egerton University",
    "Egyptian Chaineese University": "Egyptian Chinese University",
    "Eiilm": "EIILM University",
    "Ekiti State University": "Ekiti State University",
    "Ekiti State University Ado Ekiti": "Ekiti State University",
    "Enugu State University": "Enugu State University",
    "Enugu State University Of Science And Technology": "Enugu State University Of Science And Technology",
    "Fast Nuces": "FAST-NUCES",
    "Fast -Nuces": "FAST-NUCES",
    "Federal University Of Agriculture Abeokuta": "Federal University Of Agriculture",
    "Federal University Of Agriculture Makurdi": "Federal University Of Agriculture",
    "Federal University Of Agriculture Makurdi Benue State": "Federal University Of Agriculture",
    "Federal University Of Lafia": "Federal University Of Lafia",
    "Federal University Of Maiduguri": "Federal University Of Maiduguri",
    "Federal University Of Petroleum Resource Effurun Delta State": "Federal University Of Petroleum Resources",
    "Federal University Of Petroleum Resources": "Federal University Of Petroleum Resources",
    "Federal University Of Petroleum Resources Effurun": "Federal University Of Petroleum Resources",
    "Federal University Of Petroleum University": "Federal University Of Petroleum Resources",
    "Federal University Of Technology Akure": "Federal University Of Technology",
    "Federal University Of Technology Akure Nigeria": "Federal University Of Technology",
    "Federal University Of Technology Minna": "Federal University Of Technology",
    "Federal University Of Technology Minna Nigeria": "Federal University Of Technology",
    "Federal University Of Technology Owerri": "Federal University Of Technology",
    "Federal University Of Technology Owerri Nigeria": "Federal University Of Technology",
    "Federal University Of Tecnology Owerri Imo Nigeria": "Federal University Of Technology",
    "Federal University Oye Ekiti": "Federal University Oye Ekiti",
    "Federal University Oye Ekiti.": "Federal University Oye Ekiti",
    "Federal University Oye-Ekiti Ekiti": "Federal University Oye Ekiti",
    "Federal University Wukari": "Federal University Wukari",
    "Federal University Wukari Taraba": "Federal University Wukari",
    "Federal University Wukari Taraba State Nigeria": "Federal University Wukari",
    "Fiem": "FIEM University",
    "Fiitjee": "FIITJEE",
    "Fourah Bay College University Of Sierra Leone": "Fourah Bay College, University Of Sierra Leone",
    "Francis Howell": "Francis Howell",
    "Francis Howell High School": "Francis Howell",
    "Francis Howell North High School": "Francis Howell",
    "Fsc Medical": "FSC Medical",
    "Funai College": "FUNAI College",
    "Fuoye": "Federal University Oye-Ekiti",
    "Future Institute Of Engineering And Management": "Future Institute Of Engineering And Management",
    "Fuuast": "Federal University of Agriculture, Abeokuta",
    "G Naraynamma Institute Of Technology And Science": "G Naraynamma Institute Of Technology And Science",
    "G.Narayanamma Institute Of Technology Ans Sciences": "G Naraynamma Institute Of Technology And Science",
    "G. B. Pant Govt Engg College New Delhi": "G B Pant Government Engineering College New Delhi",
    "G.D. Goenka Global School": "G D Goenka Global School",
    "G.L.A University Mathura": "GLA University",
    "Gla University": "GLA University",
    "G.M Institute Of Technology": "GM Institute Of Technology",
    "Gm Institute Of Technology": "GM Institute Of Technology",
    "Galgotias University": "Galgotias University",
    "Gambia High School": "Gambia High School",
    "Ganapathy Engineering College": "Ganapathy Engineering College",
    "Gandhi Institute Of Technology And Management": "Gandhi Institute Of Technology And Management",
    "Gandhinagar Institute Of Technology": "Gandhinagar Institute Of Technology",
    "Ganga Institute Of Technology And Management": "Ganga Institute Of Technology And Management",
    "Gannon University": "Gannon University",
    "Ganpat University": "Ganpat University",
    "Garden City University College": "Garden City University College",
    "Garissa University": "Garissa University",
    "Gates Institute Of Technology": "Gates Institute Of Technology",
    "Gauhati University": "Gauhati University",
    "Gayatri Vidya Parishad College Of Engineeirng": "Gayatri Vidya Parishad College Of Engineering",
    "Gayatri Vidya Parishad College Of Engineering": "Gayatri Vidya Parishad College Of Engineering",
    "Gayatri Vidya Parishad College Of Engineering (Autonomous)": "Gayatri Vidya Parishad College Of Engineering",
    "Gc Women University Sialkot": "GC Women University Sialkot",
    "Gcuf": "Government College University Faisalabad",
    "Gds": "GDS Institute",
    "Gec Rajkot": "GEC Rajkot",
    "Geeta University": "Geeta University",
    "Geethanjali Collage Of Engineering And Technology": "Geethanjali College Of Engineering And Technology",
    "Geethanjali College Of Engineering And Technology": "Geethanjali College Of Engineering And Technology",
    "Gems New Millennium School Al Khail": "GEMS New Millennium School Al Khail",
    "George Washington High School": "George Washington High School",
    "German University In Cairo": "German University In Cairo",
    "Ghana Institute Of Management And Public Administration": "Ghana Institute Of Management And Public Administration",
    "Ghatail Cantonment Public School And College": "Ghatail Cantonment Public School And College",
    "Ghazi University Dgkhan": "Ghazi University Dgkhan",
    "Giatm": "GIATM",
    "Gift University": "Gift University",
    "Gimt": "GIMT",
    "Gitam": "GITAM University",
    "Gitam University Hyderabad Business School": "GITAM University",
    "GITAM University": "GITAM University",
    "Global Academy Of Technology": "Global Academy Of Technology",
    "Global Institute Of Technology And Management": "Global Institute Of Technology And Management",
    "Global Wealth University": "Global Wealth University",
    "Global Wealth University Togo": "Global Wealth University",
    "Gndg Commerce College": "GNDG Commerce College",
    "Gndit": "GND Institute of Technology",
    "Gndu": "Guru Nanak Dev University",
    "Gokaraju Rangaraju Institute Of Engineering And Technology": "Gokaraju Rangaraju Institute Of Engineering And Technology",
    "Gombe State University": "Gombe State University",
    "Gombe State University Tudun Wada Gombe State": "Gombe State University",
    "Gomel High School": "Gomel High School",
    "Google Digital Garage": "Google Digital Garage",
    "Goverment Polytechnic For Girls Ahemedabad": "Government Polytechnic For Girls Ahmedabad",
    "Government Ambala Muslim Graduate College": "Government Ambala Muslim Graduate College",
    "Government City College": "Government City College",
    "Government College Of Engineering And Research Avasari Pune": "Government College Of Engineering And Research Avasari Pune",
    "Government College Of Engineering Chandrapur": "Government College Of Engineering Chandrapur",
    "Government College University Faisalabad Layyah Campus": "Government College University Faisalabad",
    "Government Engineering College Thrissur": "Government Engineering College Thrissur",
    "Government Polytechnic Ahmedabad": "Government Polytechnic Ahmedabad",
    "Government Polytechnic Nashik": "Government Polytechnic Nashik",
    "Government Science College": "Government Science College",
    "Government Secondary Schools Tudun Wada": "Government Secondary Schools Tudun Wada",
    "Government Sheikh Mujibur Rahman College Tungipara Gopalganj": "Government Sheikh Mujibur Rahman College",
    "Government Stanley Medical College": "Government Stanley Medical College",
    "Govt Graduate Science College Multan": "Government Graduate Science College Multan",
    "Govt National College": "Government National College",
    "Govt School": "Government School",
    "Govt. College University Faisalabad": "Government College University Faisalabad",
    "Govt. Engineering College Hassan": "Government Engineering College Hassan",
    "Govt. Model Engineering College": "Government Model Engineering College",
    "Govt.Boys Higher Secondary School": "Government Boys Higher Secondary School",
    "Graduated From High School": "High School",
    "Graphic Era Deemed To Be University": "Graphic Era Deemed To Be University",
    "Graphic Era Hill University": "Graphic Era Hill University",
    "Graphic Era Hill University Dehradun": "Graphic Era Hill University",
    "Great Faith Alive School": "Great Faith Alive School",
    "Great Lakes Institute Of Management": "Great Lakes Institute Of Management",
    "Great Lakes Institute Of Technology": "Great Lakes Institute Of Technology",
    "Great Learning": "Great Learning",
    "Greater Noida Institute Of Technology": "Greater Noida Institute Of Technology",
    "Gsss Institute Of Engineering And Technology For Women": "GSSS Institute Of Engineering And Technology For Women",
    "Gudlavalleru Engineering College": "Gudlavalleru Engineering College",
    "Gujarat Adani Institute Of Medical Sciences Bhuj": "Gujarat Adani Institute Of Medical Sciences Bhuj",
    "Gujarat Technological University": "Gujarat Technological University",
    "Gujranwala Guru Nanak Institute Of Management And Technology": "Gujranwala Guru Nanak Institute Of Management And Technology",
    "Gulbarga College Of Pharmacy": "Gulbarga College Of Pharmacy",
    "Guru Gobind Singh Educational Society'S Technical Campus": "Guru Gobind Singh Educational Society's Technical Campus",
    "Guru Gobind Singh Indraprastha University": "Guru Gobind Singh Indraprastha University",
    "Guru Nanak Dev University": "Guru Nanak Dev University",
    "Guru Nanak Dev University College Verka": "Guru Nanak Dev University",
    "Guru Nanak Girls College": "Guru Nanak Girls College",
    "Guru Nanak Institute Of Technology": "Guru Nanak Institute Of Technology",
    "Guru Nanak Institute Of Technology Nagpur": "Guru Nanak Institute Of Technology",
    "Gurukul Kangri Deemed To Be University": "Gurukul Kangri Vishwavidyalaya",
    "Gurukul Kangri Vishwavidyalaya": "Gurukul Kangri Vishwavidyalaya",
    "Gurunanak Institute Of Technical Campus": "Gurunanak Institute Of Technical Campus",
    "Gurunanak Institutions Of Technical Campus": "Gurunanak Institute Of Technical Campus",
    "Guy M. Sconzo Early College High School": "Guy M. Sconzo Early College High School",
    "Gymnasium #132 (Almaty Kz)": "Gymnasium #132 Almaty",
    "H. N. B Garhwal University": "H N B Garhwal University",
    "H.R. College Of Commerce And Economics": "HR College Of Commerce And Economics",
    "Habib University": "Habib University",
    "Hailey College Of Banking And Finance University Of The Punjab": "Hailey College Of Banking And Finance, University Of The Punjab",
    "Hamdard University": "Hamdard University",
    "Hamdard University Karachi": "Hamdard University",
    "Helwan University": "Helwan University",
    "Heritage Experiential Learning School": "Heritage Experiential Learning School",
    "Heritage Xperiential Learning School": "Heritage Experiential Learning School",  
    "Heritage Institute Of Technology": "Heritage Institute Of Technology",
    "Himanchal Pradesh University": "Himachal Pradesh University",
    "Hindu College Pg Courses": "Hindu College Pg Courses",
    "Hindustan Institute Of Technology And Science": "Hindustan Institute Of Technology and Science",
    "Hindustan Institution Of Technology And Science": "Hindustan Institute Of Technology and Science",  # typo correction
    "Hindusthan College Of Engineering And Technology": "Hindustan College Of Engineering And Technology",  # typo correction
    "Hkbk College Of Engineering": "HKBK College Of Engineering",
    "Hmr National Pu College": "HMR National PU College",
    "Hnbgu": "Hemwati Nandan Bahuguna Garhwal University",
    "Ho Technical University": "Ho Technical University",
    "Holkar Science College": "Holkar Science College",
    "Holy Cross College": "Holy Cross College",
    "Horizon Gautam Buddha Secondary School": "Horizon Gautam Buddha Secondary School",
    "Houdegbe North America University Benin.": "Houdegbe North America University Benin",
    "Humber": "Humber College",
    "I.K Gujral Punjab Technical University": "I.K. Gujral Punjab Technical University",
    "Iba Karachi - Institute Of Business Administration": "Institute of Business Administration",
    "Iba Sukkur": "Institute of Business Administration",
    "Ibadan Polytechnic": "Ibadan Polytechnic",
    "Icat Design And Media College": "ICAT Design and Media College",
    "Icfai Business School": "ICFAI Business School",
    "Icfai University": "ICFAI University",
    "Ict Academy Of Kerala": "ICT Academy of Kerala",
    "Ideal College": "Ideal College",
    "Ideal Institute Of Engineering": "Ideal Institute Of Engineering",
    "Ifet College Of Engineering": "IFET College of Engineering",
    "Ifsc - Instituto Federal De Educacao Ciencia E Tecnologia De Santa Catarina- Campus Joinville": "IFSC - Instituto Federal de Educação, Ciência e Tecnologia de Santa Catarina, Campus Joinville",
    "Igbinedion University Okada": "Igbinedion University, Okada",
    "Igdtuw": "Indira Gandhi Delhi Technical University for Women",
    "Ignatius Ajaro University Wife Rivers State": "Ignatius Ajuru University of Education, Rivers State",
    "Ignatius Global School": "Ignatius Global School",
    "Ignou": "Indira Gandhi National Open University",
    "Iheris University Togo": "IHERIS University, Togo",
    "Ihrd": "Institute of Human Resource Development",
    "Iiit Rk Valley": "IIIT RK Valley",
    "Iiit-Rgukt Rk Valley": "IIIT RK Valley",
    "Iim Kozhikode": "Indian Institute of Management",
    "Iim Nagpur": "Indian Institute of Management",
    "Iimt University": "IIMT University",
    "Iit Chicago": "Illinois Institute of Technology",
    "Iit Delhi": "Indian Institute of Technology",
    "Iit Ism Dhanbad": "Indian Institute of Technology (ISM) Dhanbad",
    "Iit(Ism) Dhanbad": "Indian Institute of Technology (ISM) Dhanbad",
    "Ikenne Community High School": "Ikenne Community High School",
    "Ilboru High School": "Ilboru High School",
    "Illinois Institute Of Engineering": "Illinois Institute of Technology",
    "Illinois Institute Of Technolog": "Illinois Institute of Technology",
    "Illinois Institute of Technology": "Illinois Institute of Technology",
    "Illinois Institute Of Technology Chicago": "Illinois Institute of Technology",
    "Illinois Institute Of Techonology": "Illinois Institute of Technology",
    "Illinois Institution Of Technology": "Illinois Institute of Technology",
    "Illinois Intitute Of Technology": "Illinois Institute of Technology",
    "Illinos Institute Of Technology": "Illinois Institute of Technology",
    "Illinois Institute Of Technology": "Illinois Institute of Technology",
    "Illinois Math And Science Academy": "Illinois Math and Science Academy",
    "Illinois Tech University": "Illinois Institute of Technology",
    "Imarticus Learning": "Imarticus Learning",
    "Imcc College": "IMCC College",
    "Imo Polytechnic": "Imo Polytechnic",
    "Imo State College Of Nursing And Midwifery": "Imo State College of Nursing and Midwifery",
    "Imo State University": "Imo State University",
    "Imo State University Owerri": "Imo State University",
    "Imo State University Teaching Hospital Orlu": "Imo State University",
    "Ims Engineering College Ghaziabad": "IMS Engineering College, Ghaziabad",
    "Indian Institute Of Information Technology Allahabad": "Indian Institute of Information Technology",
    "Indian Institute Of Information Technology Bhopal": "Indian Institute of Information Technology",
    "Indian Institute Of Information Technology Nuzvid": "Indian Institute of Information Technology",
    "Indian Institute Of Public Health": "Indian Institute of Public Health",
    "Indian Institute Of Public Health Delhi": "Indian Institute of Public Health",
    "Indian Institute Of Science Education And Research Bhopal (Iiser Bhopal)": "Indian Institute Of Science Education And Research",
    "Indian Institute Of Social Welfare And Business Management": "Indian Institute of Social Welfare and Business Management",
    "Indian Institute Of Technology (Iit) Mandi": "Indian Institute of Information Technology",
    "Indian Institute Of Technology Bhu Varanasi": "Indian Institute of Information Technology",
    "Indian Institute Of Technology Jodhpur (Iit Jodhpur)": "Indian Institute of Information Technology",
    "Indian Institute Of Technology Madras": "Indian Institute of Information Technology",
    "Indian Ocean University": "Indian Ocean University",
    "Indo Danish Tool Room": "Indo Danish Tool Room",
    "Indus International School Pune": "Indus International School, Pune",
    "Informatics Academy": "Informatics Academy",
    "Institute In Emerging Technology": "Institute in Emerging Technology",
    "Institute Of Aeronautical Engineering": "Institute of Aeronautical Engineering",
    "Institute Of Business Administration": "Institute of Business Administration",
    "Institute Of Business Administration (Iba) Karachi": "IBA Karachi",
    "Institute Of Charted Accountance Of India": "Institute of Chartered Accountancy of India",
    "Institute Of Engineering And Technology": "Institute of Engineering and Technology",
    "Institute Of Innovation In Technology And Management": "Institute of Innovation in Technology and Management",
    "Institute Of International Relations": "Institute of International Relations",
    "Institute Of Management And Technology": "Institute of Management and Technology",
    "Institute Of Management Science Peshawar": "Institute of Management Sciences, Peshawar",
    "Institute Of Management Sciences": "Institute of Management Sciences",
    "Institute Of Pharmaceutical And Educational Research Center Wardha Maharashtra": "Institute of Pharmaceutical and Educational Research Center, Wardha",
    "Institute Of Professional Psychology Bahria University": "Institute of Professional Psychology, Bahria University",
    "Institute Superieur Des Sciences Appliques Et Technologies De Sousse": "Institut Supérieur des Sciences Appliquées et Technologies de Sousse",
    "Institutes Of Public Administration And Management": "Institute of Public Administration and Management",
    "International Business Management Institute Berlin Germany": "International Business Management Institute, Berlin",
    "International Institute Of Information Technology Bangalore": "IIIT Bangalore",
    "International Institute Of Information Technology Pune": "IIIT Pune",
    "International Institute Of Professional Studies Davv": "IIPS DAVV",
    "International Islamic University Chittagong": "International Islamic University Chittagong",
    "International Islamic University Islamabad": "International Islamic University Islamabad",
    "International Open University": "International Open University",
    "International School For Liberal Arts": "International School for Liberal Arts",
    "International School Of Management Excellence": "International School of Management Excellence",
    "Iobm": "Institute of Business Management (IoBM)",
    "Iowa State University": "Iowa State University",
    "Ips College": "IPS College",
    "Ips Dehradun": "IPS Dehradun",
    "Isa University": "ISA University",
    "Isb&M Pune": "ISB&M Pune",
    "Isbm College Of Engineering": "ISBM College of Engineering",
    "Isbm&Coe": "ISBM College of Engineering",
    "Islamia College Peshawar": "Islamia College Peshawar",
    "Islamic Azad University": "Islamic Azad University",
    "Islamic University": "Islamic University",
    "Islington College": "Islington College",
    "Ismailia Secondary Governmental Language School": "Ismailia Secondary Governmental Language School",
    "Istanbul Bilgi University": "Istanbul Bilgi University",
    "Iter": "ITER",
    "Itm Sls Baroda University": "ITM SLS Baroda University",
    "Itm Sls University": "ITM SLS University",
    "Iub": "IUB",
    "J.B. Institute Of Engineering & Technology": "J.B. Institute of Engineering & Technology",
    "J.V. Jain Degree College Saharanpur": "J.V. Jain Degree College, Saharanpur",
    "Jagannath University": "Jagannath University",
    "Jagran College Of Arts Science And Commerce": "Jagran College of Arts, Science and Commerce",
    "Jagran College Of Arts Sciences And Commerce": "Jagran College of Arts, Science and Commerce",
    "Jai Bharat Art'S And Science College": "Jai Bharat Arts and Science College",
    "Jain University": "Jain University",
    "James Hope College": "James Hope College",
    "Jamia Hamdard": "Jamia Hamdard",
    "Jamia Hamdard New Delhi": "Jamia Hamdard",
    "Jamia Hamdard University": "Jamia Hamdard",
    "Jamia Millia Islamia": "Jamia Millia Islamia",
    "Jammu College Of Physiotherapy": "Jammu College of Physiotherapy",
    "Jawaharlal Nehru Medical College": "Jawaharlal Nehru Medical College",
    "Jawaharlal Nehru New College Of Engineering Shivamogga": "Jawaharlal Nehru New College of Engineering",
    "Jawaharlal Nehru Technological Universities Vizianagaram": "Jawaharlal Nehru Technological University",
    "Jawaharlal Nehru Technological University": "Jawaharlal Nehru Technological University",
    "Jawaharlal Nehru Technological University Hyderabad": "Jawaharlal Nehru Technological University",
    "Jawaharlal Nehru Technological University Hyderabad India": "Jawaharlal Nehru Technological University",
    "Jawaharlal Nehru Technological University Of Hyderabad": "Jawaharlal Nehru Technological University",
    "Jawaharlal Technological University": "Jawaharlal Nehru Technological University",
    "Jntuh": "Jawaharlal Nehru Technological University",
    "Jb Institute Of Engineering And Technology": "J.B. Institute of Engineering & Technology",
    "Jb Institute Of Enigineering And Technology": "J.B. Institute of Engineering & Technology",
    "Jecrc Udml College Of Engineering": "JECRC UDML College of Engineering",
    "Jeddah Knowledge International School": "Jeddah Knowledge International School",
    "Jefferson City High School": "Jefferson City High School",
    "Jennings Senior High": "Jennings Senior High School",
    "Jerce University": "Jerce University",
    "Jhenaidah Cadet College": "Jhenaidah Cadet College",
    "Jiet": "JIET",
    "Jigawa State Polytechnic Dutse": "Jigawa State Polytechnic, Dutse",
    "Jijamata Arts And Science Commerce College Nandurbar": "Jijamata Arts and Science Commerce College, Nandurbar",
    "Jinnah University For Women": "Jinnah University for Women",
    "Jiwaji University": "Jiwaji University",
    "Jkuat": "Jomo Kenyatta University of Agriculture and Technology",
    "Jnct College Bhopal (M.P.)": "JNCT College, Bhopal",
    "Jntu": "Jawaharlal Nehru Technological University",
    "Jntu Kakinada": "Jawaharlal Nehru Technological University, Kakinada",
    "Jntuacek": "JNTUACEK",
    "Jntuh Ucmh": "JNTUH UCMH",
    "Jntuhceh": "JNTUH CEH",
    "Jntuk": "Jawaharlal Nehru Technological University, Kakinada",
    "Jntuk-University College Of Engineering Vizianagaram": "JNTUK University College of Engineering, Vizianagaram",
    "Johns Creek High School": "Johns Creek High School",
    "Johtech High School": "Johtech High School",
    "Jomo Kenyatta University Of Agriculture And Technology": "Jomo Kenyatta University of Agriculture and Technology",
    "Joshua": "Joshua Institute",
    "Jspm Jaywantrao Sawant College Of Engineering": "JSPM Jaywantrao Sawant College of Engineering",
    "Juet Guna": "JUET Guna",
    "K L Deemed To Be University": "K.L. Deemed to be University",
    "K L University": "K.L. University",
    "K S Rangasamy College Of Technology": "K.S. Rangasamy College of Technology",
    "K. D. K College Of Engineering": "K.D.K College of Engineering",
    "K.C College": "K.C. College",
    "K.L.E K.F.Patil Institute Of Business Administration": "K.L.E. K.F. Patil Institute of Business Administration",
    "K.S.G.M Collage": "K.S.G.M College",
    "K.S.Rangasamy College Of Technology": "K.S. Rangasamy College of Technology",
    "Kadi Sarva Vishwavidyalaya": "Kadi Sarva Vishwavidyalaya",
    "Kaduna Polytechnic": "Kaduna Polytechnic",
    "Kaduna State University": "Kaduna State University",
    "Kafr Elsheikh": "Kafr Elsheikh University",
    "Kakatiya Institute Of Technology": "Kakatiya Institute of Technology",
    "Kakatiya Institute Of Technology And Sciences": "Kakatiya Institute of Technology and Sciences",
    "Kakatiya Institute Of Technology Of Science": "Kakatiya Institute of Technology and Sciences",
    "Kakatiya University": "Kakatiya University",
    "Kakinada Institute Of Engineering And Technology": "Kakinada Institute of Engineering and Technology",
    "Kakinada Institute Of Engineering And Technology For Women'S": "Kakinada Institute of Engineering and Technology",
    "Kalaheo High School": "Kalaheo High School",
    "Kaleab Secondary And Preparatory School": "Kaleab Secondary and Preparatory School",
    "Kalinga Institute Of Industrial Technology": "Kalinga Institute of Industrial Technology",
    "Kalinga Institute Of Information Technology": "Kalinga Institute of Information Technology",
    "Kalol Institute Of Technology & Research Center": "Kalol Institute of Technology & Research Center",
    "Kalpataru Institute Of Technology": "Kalpataru Institute of Technology",
    "Kalyani Govt. Engineering College": "Kalyani Government Engineering College",
    "Kamla Nehru Mahavidhyalay Nagpur": "Kamla Nehru Mahavidyalay, Nagpur",
    "Kamuzu University Of Health Sciences": "Kamuzu University of Health Sciences",
    "Kano University Of Science And Technology Wudil": "Kano University of Science and Technology, Wudil",
    "Karachi Institute Of Economics And Technology": "Karachi Institute of Economics and Technology",
    "Karakoram International University (Gb)": "Karakoram International University",
    "Karakurum International University Gilgit": "Karakoram International University",
    "Karunya Institute Of Technology And Sciences": "Karunya Institute of Technology and Sciences",
    "Karunya Institution Technology And Sciences": "Karunya Institute of Technology and Sciences",
    "Karunya University": "Karunya University",
    "Kasiya College": "Kasiya College",
    "Kavikulguru Institute Of Technology And Science Ramtek": "Kavikulguru Institute of Technology and Science, Ramtek",
    "Kbt College Of Engineering": "KBT College of Engineering",
    "Kebbi State University Of Science And Technology": "Kebbi State University of Science and Technology",
    "Kebbi State University Of Science And Technology Aliero": "Kebbi State University of Science and Technology, Aliero",
    "Kedrus Academy": "Kedrus Academy",
    "Kelkar College": "Kelkar College",
    "Kenya Railways Corporation": "Kenya Railways Corporation",
    "Kenya Utalii College": "Kenya Utalii College",
    "Kenyatta University": "Kenyatta University",
    "Keshav Memorial Institute Of Commerce And Science": "Keshav Memorial Institute of Commerce and Science",
    "Keshav Memorial Institute Of Technology": "Keshav Memorial Institute of Technology",
    "Khalsa College For Women Amritsar": "Khalsa College for Women, Amritsar",
    "Kharkiv National University Of Radioelectronics": "Kharkiv National University of Radioelectronics",
    "Khawaja Fareed University Of Information And Technology": "Khawaja Fareed University of Information and Technology",
    "Kiambu Institute Of Science & Technology": "Kiambu Institute of Science and Technology",
    "Kiet Group Of Institusions": "KIET Group of Institutions",
    "King'S College Budo": "King's College Budo",
    "Kingston Engineering College": "Kingston Engineering College",
    "Kips College": "KIPS College",
    "Kirinyaga University": "Kirinyaga University",
    "Kisii University": "Kisii University",
    "Kit College Of Engineering": "KIT College of Engineering",
    "Kitengela Technical Institute": "Kitengela Technical Institute",
    "Kkr And Ksr Institute Of Technology And Sciences": "KKR & KSR Institute of Technology and Sciences",
    "Kl Deemed To Be University": "K.L. Deemed to be University",
    "Kl University": "K.L. University",
    "Kluniversity": "K.L. University",
    "Kml Vatsalya College Vidisha": "KML Vatsalya College, Vidisha",
    "Knust": "Kwame Nkrumah University of Science and Technology (KNUST)",
    "Knutsford University College": "Knutsford University College",
    "Kodada Institute Of Technology & Science For Women": "Kodada Institute of Technology & Science for Women",
    "Koforidua Technical University": "Koforidua Technical University",
    "Kogi State University": "Kogi State University",
    "Kogi State University Anyigba": "Kogi State University, Anyigba",
    "Kogi State University Ayingba": "Kogi State University, Anyigba",
    "Kokomo High School": "Kokomo High School",
    "Kolhan University": "Kolhan University",
    "Koringa College Of Pharmacy": "Koringa College of Pharmacy",
    "Kr Mangalam": "KR Mangalam University",
    "Krishna Institute Of Medical Sciences Deemed University Karad": "Krishna Institute of Medical Sciences, Deemed University, Karad",
    "Krishna Menon Memorial Women'S Govt College": "Krishna Menon Memorial Women's Govt College",
    "Kthm College": "KTHM College",
    "Kuban State Medical University": "Kuban State Medical University",
    "Kumanu University Nainital Uttarakhand": "Kumanu University, Nainital, Uttarakhand",
    "Kumaraguru College Of Technology": "Kumaraguru College of Technology",
    "Kumasi Technical University": "Kumasi Technical University",
    "Kuriakose Elias English Medium School": "Kuriakose Elias English Medium School",
    "Kurukshetra University": "Kurukshetra University",
    "Kwame Nkrumah University Of Science And Technology": "Kwame Nkrumah University of Science and Technology",
    "Kwame Nkrumah University Of Science And Technology (Knust)": "Kwame Nkrumah University of Science and Technology",
    "Kwame Nkrumah University Of Science And Technology Knust": "Kwame Nkrumah University of Science and Technology",
    "Kwara State Polytechnic": "Kwara State Polytechnic",
    "Kwara State University": "Kwara State University",
    "Kwara State University Malete Nigeria.": "Kwara State University",
    "Kwara State University Malete.": "Kwara State University",
    "Kebbi State University of Science and Technology": "Kebbi State University of Science and Technology",
    "Kebbi State University of Science and Technology, Aliero": "Kebbi State University of Science and Technology",
    "Kyethivo Girls": "Kyethivo Girls",
    "Ladoke Akintola University Of Technology": "Ladoke Akintola University of Technology",
    "Ladoke Akintola University Of Technology Ogbomoso": "Ladoke Akintola University of Technology",
    "Ladoke Akintola University Of University": "Ladoke Akintola University of Technology",
    "Ladue Horton Watkins High": "Ladue Horton Watkins High School",
    "Lady Doak College": "Lady Doak College",
    "Lagos Business School": "Lagos Business School",
    "Lagos State Polytechnic": "Lagos State Polytechnic",
    "Lagos State University": "Lagos State University",
    "Lagos State University (Lasu)": "Lagos State University",
    "Lahore College For Women University": "Lahore College for Women University",
    "Lake Central High School": "Lake Central High School",
    "Lake Nona High School": "Lake Nona High School",
    "Landmark University": "Landmark University",
    "Lasbela University Of Agriculture Water & Marine Sciences": "Lasbela University of Agriculture, Water & Marine Sciences",
    "Leading University Sylhet": "Leading University, Sylhet",
    "Lebanese University": "Lebanese University",
    "Lgs 55Main": "LGS 55 Main",
    "Liaquat University Of Medical And Health Sciences Jamshoro": "Liaquat University of Medical and Health Sciences, Jamshoro",
    "Libyan International Medical University": "Libyan International Medical University",
    "Lideta Catholic Cathedral School": "Lideta Catholic Cathedral School",
    "Limkokwing University Of Creative Technology": "Limkokwing University of Creative Technology",
    "Lincoln Way West": "Lincoln Way West High School",
    "Little Angel'S College": "Little Angels College",
    "Little Flower Junior College": "Little Flower Junior College",
    "Liverpool International College": "Liverpool International College",
    "Liverpool John Moores University": "Liverpool John Moores University",
    "Lj Institute Of Management Studies": "LJ Institute of Management Studies",
    "LJ University": "LJ University",
    "Lokmanya Tilak College Of Engineering": "Lokmanya Tilak College of Engineering",
    "Lokmanya Tilak College Of Engineering Koperkhairne": "Lokmanya Tilak College of Engineering",
    "London Metropolitan University": "London Metropolitan University",
    "Lords Instituite Of Engineering And Technology": "Lords Institute of Engineering and Technology",
    "Lotus Valley International School": "Lotus Valley International School",
    "Lotus Valley International School Noida": "Lotus Valley International School",
    "Lotus Valley International School": "Lotus Valley International School",
    "Lotus Valley International School, Noida": "Lotus Valley International School",
    "Lotus Valley International School, Noida, Uttar Pradesh": "Lotus Valley International School",
    "Lotus Valley International": "Lotus Valley International School",
    "Lotus Valley International School Noida Uttar Pradesh": "Lotus Valley International School",
    "Lasu": "Lagos State University",
    "Lau": "Ladoke Akintola University of Technology",
    "Lautech": "Ladoke Akintola University of Technology",
    "Lcwu": "Lahore College for Women University",
    "Lj University": "LJ University",
    "Ljims": "LJ Institute of Management Studies",
    "Lnit Srinagar": "National Institute of Technology Srinagar",
    "Lgs 55main": "Lahore Grammar School",
    "London Metropolitan University Uk": "London Metropolitan University",
    "Loveland Highschool": "Loveland High School",
    "Lovely Professional University": "Lovely Professional University",
    "Loyola College": "Loyola College",
    "Lpu": "Lovely Professional University",
    "Lucknow University": "University of Lucknow",
    "Lycee Ibn Abi Dhief": "Lycee Ibn Abi Dhief",
    "Lyceum Of Social And Natural Profile No. 303": "Lyceum of Social and Natural Profile",
    "Ladoke Akintola University of Technology": "Ladoke Akintola University of Technology",
    "Ladoke Akintola University of Technology, Ogbomoso": "Ladoke Akintola University of Technology",
    "Lotus Valley International School": "Lotus Valley International School",
    "Lotus Valley International School, Noida": "Lotus Valley International School",
    "Lotus Valley International School, Noida, Uttar Pradesh": "Lotus Valley International School",
    "M..S.Bidve Engineering College Latur": "M.S. Bidve Engineering College, Latur",
    "M.G University": "Mahatma Gandhi University",
    "M.Tech": "M.Tech Program",
    "Maac": "MAAC Institute",
    "Madanapalle Institute Of Technology And Science": "Madanapalle Institute of Technology and Science",
    "Madanapalli Institute Of Technology & Science": "Madanapalle Institute of Technology and Science",
    "Madanappale Institute Of Technology And Science": "Madanapalle Institute of Technology and Science",
    "Madras Christian College": "Madras Christian College",
    "Madras University": "University of Madras",
    "Magadh University": "Magadh University",
    "Maharaja Agrasen Institute Of Technology": "Maharaja Agrasen Institute of Technology",
    "Maharana Pratap Engineering College": "Maharana Pratap Engineering College",
    "Maharishi Markandeshwar University": "Maharishi Markandeshwar University",
    "Maharshi Dayanand University": "Maharshi Dayanand University",
    "Mahatma Gandhi University": "Mahatma Gandhi University",
    "Makerere University": "Makerere University",
    "Malaviya National Institute Of Technology Jaipur": "Malaviya National Institute of Technology, Jaipur",
    "Malla Reddy College Of Engineering": "Malla Reddy College of Engineering and Technology",
    "Malla Reddy College Of Engineering And Technology": "Malla Reddy College of Engineering and Technology",
    "Malla Reddy Engineering College": "Malla Reddy College of Engineering and Technology",
    "Malla Reddy Engineering College For Women": "Malla Reddy College of Engineering and Technology",
    "Mallareddy Engineering College": "Malla Reddy College of Engineering and Technology",
    "Malla Reddy University": "Malla Reddy University",
    "Malnad College Of Engineering": "Malnad College of Engineering",
    "Manaratul Uloom": "Manaratul Uloom",
    "Mangalore University": "Mangalore University",
    "Manglore Institute Of Technology &Engineering": "Mangalore Institute of Technology & Engineering",
    "Manipal Institute Of Technology": "Manipal Institute of Technology",
    "Manipal University": "Manipal University",
    "Manipal University Dubai": "Manipal University",
    "Manipal University Jaipur": "Manipal University",
    "Mankessim Senior High Technical School": "Mankessim Senior High Technical School",
    "Mansoura University": "Mansoura University",
    "Marathwada Mitra Mandal'S College Of Engineering": "Marathwada Mitra Mandal's College of Engineering",
    "Mariani College": "Mariani College",
    "Mario International School": "Mario International School",
    "Marri Laxman Reddy": "Marri Laxman Reddy Institute",
    "Marri Laxman Reddy Institute Of Pharmacy": "Marri Laxman Reddy Institute of Pharmacy",
    "Marri Laxman Reddy Institute Of Technology And Management": "Marri Laxman Reddy Institute of Technology and Management",
    "Marwadi University": "Marwadi University",
    "Maryam Abacha American University Of Nigeria": "Maryam Abacha American University of Nigeria",
    "Maseno University": "Maseno University",
    "Master Ji Degree College": "Master Ji Degree College",
    "Mata Sundri Sen.Sec Public School": "Mata Sundri Senior Secondary Public School",
    "Mathematics And Physics": "Mathematics and Physics",
    "Matoshri College Of Engineering": "Matoshri College of Engineering",
    "Matrusri Engineering College": "Matrusri College of Engineering",
    "Maturi Venkata Subba Rao Engineering College": "Maturi Venkata Subba Rao Engineering College",
    "Maulana Abul Kalam Azad University Of Technology": "Maulana Abul Kalam Azad University of Technology",
    "Md Shad Mahmud": "Md Shad Mahmud",
    "Mds University": "MDS University",
    "Mds University Ajmer Rajasthan": "MDS University",
    "Meghnad Saha Institute Of Technology": "Meghnad Saha Institute of Technology",
    "Mehraj": "Mehraj",
    "Mehran Uet Szab Campus Khairpur Mir'S Sindh Pakistan": "Mehran UET SZAB Campus, Khairpur, Sindh, Pakistan",
    "Mehran Uniiversity Of Engineering And Technology Jamshoro Pakistan": "Mehran University of Engineering and Technology",
    "Mehran University": "Mehran University of Engineering and Technology",
    "Mehran University Jamshoro Sindh Pakistan": "Mehran University of Engineering and Technology",
    "Mehran University Of Engineering And Technology": "Mehran University of Engineering and Technology",
    "Mercy Businesses School": "Mercy Business School",
    "Mes College Of Engineering Kuttippuram": "MES College of Engineering, Kuttippuram",
    "Mes Ponnani College": "MES Ponnani College",
    "Metea Valley High School": "Metea Valley High School",
    "Methodist College Of Engineering And Technology": "Methodist College of Engineering and Technology",
    "Methodist Engineering Collge Ou": "Methodist Engineering College OU",
    "Methodist University": "Methodist University",
    "Metropolitan School Of Business And Management": "Metropolitan School of Business and Management",
    "Mgit": "MGIT",
    "Mgm High School": "MGM High School",
    "Mgm'S College Of Engineering Nanded": "MGM's College of Engineering, Nanded",
    "Mian Faizan": "Mian Faizan",
    "Michael Okpara University Of Agriculture": "Michael Okpara University of Agriculture",
    "Michael Okpara University Of Agriculture Umudike": "Michael Okpara University of Agriculture",
    "Michael Okpara University Of Agriculture Umudike Abia State": "Michael Okpara University of Agriculture",
    "Micheal Okpara University Of Agriculture Umudike": "Michael Okpara University of Agriculture",
    "Middlesex University": "Middlesex University",
    "Midlands State University": "Midlands State University",
    "Millennium National School": "Millennium National School",
    "Minhaj University Lahore": "Minhaj University, Lahore",
    "Miranda House University Of Delhi": "Miranda House, University of Delhi",
    "Misr International University": "Misr International University",
    "Mission San Jose High School": "Mission San Jose High School",
    "Mit": "MIT",
    "Mit - Wpu": "MIT WPU",
    "Mit Arts Commers And Science College Alandi": "MIT Arts, Commerce and Science College, Alandi",
    "Mit World Peace University": "MIT World Peace University",
    "Mit Wpu": "MIT WPU",
    "Mit-Wpu": "MIT WPU",
    "Mjcet": "MJ College of Engineering and Technology",
    "Mlr Institute Of Technology": "MLR Institute of Technology",
    "Mlritm": "MLR Institute of Technology & Management",
    "Mnr College Of Pharmacy": "MNR College of Pharmacy",
    "Model Secondary School": "Model Secondary School",
    "Model Secondary School Amokwe": "Model Secondary School",
    "Modibbo Adama University Of Technology": "Modibbo Adama University of Technology",
    "Mohammed V University In Rabat": "Mohammed V University, Rabat",
    "Monash University": "Monash University",
    "Monta Vista High School": "Monta Vista High School",
    "Morning Glory International School": "Morning Glory International School",
    "Moshi Cooperative University": "Moshi Cooperative University",
    "Moshood Abiola Polytechnic": "Moshood Abiola Polytechnic",
    "Most High College": "Most High College",
    "Mount Kenya University": "Mount Kenya University",
    "Mountain Institute Of Management And Technology": "Mountain Institute of Management and Technology",
    "Msu": "MSU",
    "Muet": "MUET",
    "Muffakham Jah College Of Engineering And Technology": "Muffakham Jah College of Engineering and Technology",
    "Mumbai University": "University of Mumbai",
    "Must": "MUST University",
    "Mustuniversity": "MUST University",
    "Mvgr College Of Engineering": "MVGR College of Engineering",
    "Mvj College Of Engineering": "MVJ College of Engineering",
    "Mvsr Engineering": "MVSR Engineering College",
    "Mvsr Engineering College": "MVSR Engineering College",
    "Mzumbe University": "Mzumbe University",
    "Nagarjuna College Of Engineering & Technology": "Nagarjuna College of Engineering & Technology",
    "Nagpur University": "Nagpur University",
    "Nahda University": "Nahda University",
    "Nalla Narsimha Reddy Group Of Institutions": "Nalla Narsimha Reddy Group of Institutions",
    "Nallamuthu Gounder Mahalingam College Pollachi": "Nallamuthu Gounder Mahalingam College, Pollachi",
    "Nanjing University": "Nanjing University",
    "Naperville Central High School": "Naperville Central High School",
    "Narasaraopeta Engineering College": "Narasaraopeta Engineering College",
    "Narayana": "Narayana",
    "Narayana Engineering College": "Narayana Engineering College",
    "National Institute Of Pharmaceutical Education And Research Hyderabad": "National Institute of Pharmaceutical Education and Research, Hyderabad",
    "National Institute Of Rehabilitation Medicine": "National Institute of Rehabilitation Medicine",
    "National Institute Of Technology Andhra Pradesh": "National Institute of Technology",
    "National Institute Of Technology Durgapur": "National Institute of Technology",
    "National Institute Of Technology Raipur": "National Institute of Technology",
    "National Institute Of Technology Silchar": "National Institute of Technology",
    "National Institute Of Technology Srinagar": "National Institute of Technology",
    "National Institute Of Technology Surathkal": "National Institute of Technology",
    "National Institute Of Technology Tiruchirapalli": "National Institute of Technology",
    "National Institute Of Technology Uttarakhand": "National Institute of Technology",
    "National Open University": "National Open University",
    "National Open University Nigeria": "National Open University",
    "National Open University Of Nigeria": "National Open University",
    "National Pg College": "National PG College",
    "National Pirogov Medical University": "National Pirogov Medical University",
    "National University": "National University",
    "National University Bangladesh": "National University",
    "National University Of Computer & Emerging Sciences": "National University of Computer & Emerging Sciences",
    "National University Of Modern Languages Islamabad (Numl)": "National University of Modern Languages, Islamabad",
    "National University Of Science And Technology": "National University of Science and Technology",
    "National University Of Science And Technology Islamabad": "National University of Science and Technology",
    "National University Of Science And Technology Nust": "National University of Science and Technology",
    "National University Of Science And Technology Pakistan": "National University of Science and Technology",
    "National University Of Sciences And Technology": "National University of Sciences and Technology",
    "National University Of Technology(Nutech)Islamabad": "National University of Technology",
    "Navoiy Presidential School": "Navoiy Presidential School",
    "Ndejje University": "Ndejje University",
    "Nduat Ayodhya India": "NDUAT, Ayodhya, India",
    "Near East University": "Near East University",
    "Ned": "NED University of Engineering and Technology",
    "Ned University Of Engineering And Technology": "NED University of Engineering and Technology",
    "Ned University Of Science & Technology": "NED University of Science and Technology",
    "Nega": "Nega",
    "Nehru Arts And Science College": "Nehru Arts and Science College",
    "Nehru Institute Of Engineering And Technology": "Nehru Institute of Engineering and Technology",
    "Nehru Institute Of Engineering And Technology Coimbatore": "Nehru Institute of Engineering and Technology, Coimbatore",
    "Nelson Mandela University": "Nelson Mandela University",
    "Nepal Law Campus": "Nepal Law Campus",
    "Nepalaya": "Nepalaya",
    "Ness Wadia": "Ness Wadia College of Commerce",
    "Netaji Subhash Engineering College": "Netaji Subhash Engineering College",
    "New Adarsh College": "New Adarsh College",
    "New Horizon College Of Engineering": "New Horizon College of Engineering",
    "New Jersey Institute Of Technology": "New Jersey Institute of Technology",
    "New York University": "New York University",
    "Niagara University": "Niagara University",
    "Nift": "National Institute of Fashion Technology (NIFT)",
    "Niger Delta University": "Niger Delta University",
    "Niger Delta University Wilberforce Island": "Niger Delta University",
    "Nigerian Army School Of Medical Sciences": "Nigerian Army School of Medical Sciences",
    "Nigerian Defence Academy, School Of Postgraduate Studies": "Nigerian Defence Academy, School of Postgraduate Studies",
    "Nigerian Defense Academy Kaduna": "Nigerian Defence Academy, Kaduna",
    "Niit": "NIIT",
    "Niit Accra": "NIIT",
    "Nile University Of Nigeria": "Nile University of Nigeria",
    "Nims University": "NIMS University",
    "Nis- Nermein Ismail Advanced Education": "NIS - Nermein Ismail Advanced Education",
    "Nit Durgapur": "NIT",
    "Nit-Agartala": "NIT",
    "Nitesh": "NITTE",
    "Nitte Meenakshi Institute Of Technology Bangalore": "Nitte Meenakshi Institute of Technology, Bangalore",
    "Nitte School Of Management": "Nitte School of Management",
    "Nituk": "NITUK",
    "Nizam Collage": "Nizam College",
    "Nkumba University": "Nkumba University",
    "Nmam Institute Of Technology": "NMAM Institute of Technology",
    "Nmamit": "NMAM Institute of Technology & Management",
    "Nmamit Nitte": "NMAM Institute of Technology & Management",
    "Nnamdi Azikiwe University": "Nnamdi Azikiwe University",
    "Nnamdi Azikiwe University Awka": "Nnamdi Azikiwe University",
    "Nnamdi Azikwe University": "Nnamdi Azikiwe University",
    "North Maharashtra University": "North Maharashtra University",
    "North Maharashtra University Jalgaon(M.H.)": "North Maharashtra University",
    "Northeastern University": "Northeastern University",
    "Nottingham Trent University": "Nottingham Trent University",
    "Nourhan Fathy": "Nourhan Fathy",
    "Nuhu Bamalli Polytechnic Zaria": "Nuhu Bamalli Polytechnic, Zaria",
    "Nusrat Senior Secondary School": "Nusrat Senior Secondary School",
    "Nust": "NUST",
    "NWIHS": "NWIHS",
    "Nxtwave": "Nxtwave",
    "Oakridge": "Oakridge International School",
    "Oau Ile Ife": "Obafemi Awolowo University, Ile-Ife",
    "Oauthc": "OAUTHC",
    "Obafemi Awolowo University": "Obafemi Awolowo University",
    "Oduduwa University Ipetumodu": "Oduduwa University, Ipetumodu",
    "Ogun State Institute Of Technology": "Ogun State Institute of Technology",
    "Olabisi Onabanjo University": "Olabisi Onabanjo University",
    "Ooltewah High School": "Ooltewah High School",
    "Open University": "Open University",
    "Oriental Institute Of Science And Technology": "Oriental Institute of Science and Technology",
    "Oriental Public School": "Oriental Public School",
    "Oriental University": "Oriental University",
    "Orisumbare Senior High School": "Orisumbare Senior High School",
    "Osmania University": "Osmania University",
    "Osun State College Of Health Technology": "Osun State College of Health Technology",
    "Osun State University": "Osun State University",
    "Outr": "OUTR",
    "P. V. G Nashik": "P.V.G. College of Engineering, Nashik",
    "P.E.S College Of Engineering": "P.E.S. College of Engineering",
    "Pace Institute Of Technology And Sciences": "PACE Institute of Technology and Sciences",
    "Pacific Academy Of Higher Education And Research": "Pacific Academy of Higher Education and Research",
    "Pacific Academy Of Higher Educational And Research University": "Pacific Academy of Higher Education and Research",
    "Padmakanya": "Padmakanya Institute",
    "Page": "Page Institute",
    "Pakistan Institute Of Engineering And Applied Sciences": "PIEAS, Pakistan",
    "Pan-Atlantic University": "Pan-Atlantic University",
    "Panimalar Engineering College": "Panimalar Engineering College",
    "Parul Institute Of Engineering And Technology": "Parul Institute of Engineering and Technology",
    "Parul University": "Parul University",
    "Patna Womens College": "Patna Women's College",
    "Patna Women'S College": "Patna Women's College",
    "Patna Women'S College Patna": "Patna Women's College",
    "Pccoer": "PCCOER",
    "Pentecost University": "Pentecost University",
    "Pentecost University College": "Pentecost University College",
    "Periyar Maniammai Institute Of Science And Technology": "Periyar Maniammai Institute of Science and Technology",
    "Petroleum Training Institute": "Petroleum Training Institute",
    "Petroleum Training Institute (Pti)": "Petroleum Training Institute",
    "Pg Program Specialized In Data Science And Machine Learning": "PG Program in Data Science and Machine Learning",
    "Pgcc": "PGCC",
    "Philippine Normal University": "Philippine Normal University",
    "Pilla Hoc College": "Pilla HOC College",
    "Pillai College": "Pillai College of Engineering",
    "Pillai College Of Engineering": "Pillai College of Engineering",
    "Pimpri Chinchwad College Of Engineering And Research College Ravet": "Pimpri Chinchwad College of Engineering and Research, Ravet",
    "Pioneer International University": "Pioneer International University",
    "Plaksha University": "Plaksha University",
    "Plano West Senior High": "Plano West Senior High School",
    "Plateau State University Bokkos": "Plateau State University, Bokkos",
    "Pmas": "PMAS",
    "Pmas Arid Agricultural University Rawalpindi": "PMAS Arid Agriculture University, Rawalpindi",
    "Pmas Arid Agriculture University Rawalpindi": "PMAS Arid Agriculture University, Rawalpindi",
    "Podar International School": "Podar International School",
    "Pondicherry University": "Pondicherry University",
    "Ponnaiyah Ramajayam Institute Of Science And Technology": "Ponnaiyah Ramajayam Institute of Science and Technology",
    "Popcorntime.Telugu@Gmail.Com": "Popcorntime Telugu Academy",
    "Potencia Student In Slu": "Potencia Student, SLU",
    "Pragati Engineering College": "Pragati Engineering College",
    "Pragati Mahavidayalaya Degree & P.G.College King Koti Hyderabad.": "Pragati Mahavidyalaya Degree & PG College, King Koti, Hyderabad",
    "Prasad V Potluri Institute Of Technology": "Prasad V. Potluri Institute of Technology",
    "Prathyusha Engineering College": "Prathyusha Engineering College",
    "Presbyterian Boys' Secondary School": "Presbyterian Boys' Secondary School",
    "Presbyterian Senior Science School": "Presbyterian Senior Science School",
    "Presidency University Bangalore": "Presidency University, Bangalore",
    "Prist University": "PRIST University",
    "Priyadarshini College Of Engineering": "Priyadarshini College of Engineering",
    "Prof Rajendra Singh University": "Prof. Rajendra Singh University",
    "Prof Ram Meghe College Engineering And Management": "Prof. Ram Meghe College of Engineering and Management",
    "Professor Jayashankar Telangana State Agricultural University": "Professor Jayashankar Telangana State Agricultural University",
    "Prudence": "Prudence Institute",
    "Psgr Krishnammal College For Women": "PSGR Krishnammal College for Women",
    "Psit": "PSIT",
    "Pt Ddu University Of Cg Under Maitri College Of Dentistry And Research Center": "Pt. DDU University of CG under Maitri College of Dentistry and Research Center",
    "Pune Institute Of Computer Technology Pune (Pict)": "Pune Institute of Computer Technology (PICT), Pune",
    "Pune University": "Savitribai Phule Pune University",
    "Pune Vidyarthi Griha'S College Of Engineering Nashik": "Pune Vidyarthi Griha's College of Engineering, Nashik",
    "Punjab College Of Science": "Punjab College of Science",
    "Punjab Group Of College": "Punjab Group of Colleges",
    "Punjab Group Of Colleges": "Punjab Group of Colleges",
    "Punjab University": "Punjab University, Lahore",
    "Punjab University Lahore": "University of the Punjab, Lahore",
    "Punjab University": "University of the Punjab, Lahore",
    "Punjab University, Lahore": "University of the Punjab, Lahore",
    "Punjabi University": "Punjabi University, Patiala",
    "Punjabi University, Patiala": "Punjabi University, Patiala",
    "Punjabi University": "Punjabi University, Patiala",
    "Punjabi University Patiala": "Punjabi University, Patiala",
    "Punjab University": "University of the Punjab, Lahore",
    "Punjab University": "University of the Punjab, Lahore",
    "Punjab University, Lahore": "University of the Punjab, Lahore",
    "Punjabi University": "Punjabi University, Patiala",
    "Punjabi University, Patiala": "Punjabi University, Patiala",
    "Purdue": "Purdue University",
    "Purdue University": "Purdue University",
    "Pvp Siddhartha Institute Of Technology": "PVP Siddhartha Institute of Technology",
    "Pyotr Mironovich Masherov Regional Lyceum": "Pyotr Mironovich Masherov Regional Lyceum",
    "Quaid E Awam University Of Engineering Science And Technology Nawabshah": "Quaid-i-Azam University, Islamabad",
    "Quaid E Azam University Islamabad": "Quaid-i-Azam University, Islamabad",
    "Quaid-I-Azam University Islamabad": "Quaid-i-Azam University, Islamabad",
    "Quantum University": "Quantum University",
    "Queen Mary University Of London": "Queen Mary University of London",
    "R V C E": "Rashtreeya Vidyalaya College of Engineering (RVCE)",
    "R. C. Patel Institute Of Technology": "R.C. Patel Institute of Technology",
    "R.C.Patel Collge": "R.C. Patel College",
    "R.H. King Academy": "R.H. King Academy",
    "Raghu Education College": "Raghu Education College",
    "Raghu Engineering College": "Raghu Engineering College",
    "Rajagiri Public School": "Rajagiri Public School",
    "Rajalakshmi Engineering College": "Rajalakshmi Engineering College",
    "Rajalakshmi Institute Of Technology": "Rajalakshmi Institute of Technology",
    "Rajalakshmi School Of Business": "Rajalakshmi School of Business",
    "Rajarambapu Institute Of Technology": "Rajarambapu Institute of Technology",
    "Rajasthan Technical University": "Rajasthan Technical University",
    "Rajdhani Higher Secondary School Bhubaneswar": "Rajdhani Higher Secondary School, Bhubaneswar",
    "Rajeev Gandhi Memorial College Of Engineering And Technology": "Rajeev Gandhi Memorial College of Engineering and Technology",
    "Rajiv Gandhi Institute Of Health Sciences": "Rajiv Gandhi Institute of Health Sciences",
    "Rajiv Gandhi Universities Of Knowledge And Technologies Rk Valley": "Rajiv Gandhi University of Knowledge and Technology, RK Valley",
    "Rajiv Gandhi University Of Knowledge And Technologies Rk Valley": "Rajiv Gandhi University of Knowledge and Technology, RK Valley",
    "Rajiv Gandhi University Of Knowledge Technologies Nuzvid": "Rajiv Gandhi University of Knowledge Technologies, Nuzvid",
    "Ram Lalit Singh Mahavidya": "Ram Lalit Singh Mahavidya",
    "Ramachandra Engineering College": "Rama Chandra Engineering College",
    "Ramadevi Women'S University Bhubaneswar": "Ramadevi Women's University, Bhubaneswar",
    "Ramjas": "Ramjas College",
    "Ramjas College": "Ramjas College",
    "Ramniranjan Anandilal Podar College Of Commerce &Economica": "Ramniranjan Anandilal Podar College of Commerce & Economics",
    "Ramrao Adik Institute Of Technology": "Ramrao Adik Institute of Technology",
    "Ranchi University": "Ranchi University",
    "Rani Durgavati Vishwavidyalaya": "Rani Durgavati Vishwavidyalaya",
    "Rani Durgawati University Jabalpur Mp": "Rani Durgavati University, Jabalpur, MP",
    "Ras El-Bar Higher Institute For Qualitative Studies And Computer": "Ras El-Bar Higher Institute for Qualitative Studies and Computer",
    "Rashtrasant Tukadoji Maharaj Nagpur University": "Rashtrasant Tukadoji Maharaj Nagpur University",
    "Ratna Junior College": "Ratna Junior College",
    "Rd Women'S University": "RD Women's University",
    "Red Sea Stem School": "Red Sea STEM School",
    "Redeemer'S University": "Redeemer's University, Nigeria",
    "Reqelford International School": "Reqelford International School",
    "Reva University": "Reva University",
    "Revelation Secondary School": "Revelation Secondary School",
    "Rewa Engineering College": "Rewa Engineering College",
    "Rg Kedia College Of Commerce": "RG Kedia College of Commerce",
    "Riphah International University Islamabad": "Riphah International University, Islamabad",
    "Rise Krishna Sai Gandhi Group Of Institutions": "Rise Krishna Sai Gandhi Group of Institutions",
    "River State University": "Rivers State University",
    "Rivers State University": "Rivers State University",
    "Rizvi College of Engineering": "Rizvi College of Engineering",
    "RK University": "RK University",
    "RMD Engineering College": "RMD Engineering College",
    "RMK Engineering College": "RMK Engineering College",
    "RMKCET": "RMK College of Engineering and Technology",
    "Rome Business School": "Rome Business School",
    "Roots International Schools": "Roots International Schools",
    "Rosary High School": "Rosary High School",
    "RR Institute of Technology": "RR Institute of Technology",
    "Rutendo High School": "Rutendo High School",
    "RVR & JC College of Engineering": "RVR & JC College of Engineering",
    "Ryan International School": "Ryan International School",
    "Rivers State University": "Rivers State University",
    "Rizvi College Of Engineering": "Rizvi College of Engineering",
    "Rk University": "RK University",
    "Rmd Engineering College": "RMD Engineering College",
    "Rmk Engineering College": "RMK Engineering College",
    "Rmkcet": "RMKCET",
    "Rr Institute Of Technology": "RR Institute of Technology",
    "Rtmnu": "Rashtrasant Tukadoji Maharaj Nagpur University",
    "Rutendo High School": "Rutendo High School",
    "Rvr & Jc College Of Engineering": "RVR & JC College of Engineering",
    "Rvr&Jc": "RVR & JC College of Engineering",
    "Ryan International School": "Ryan International School",
    "Rivers State University": "Rivers State University",
    "Rizvi College Of Engineering": "Rizvi College of Engineering",
    "Rk University": "RK University",
    "S. N. D. College Of Engineering & Research Center Yeola": "S.N.D. College of Engineering & Research Center, Yeola",
    "Sabujbagh Government College": "Sabujbagh Government College",
    "Sacred Heart College": "Sacred Heart College",
    "Sadiq Public School": "Sadiq Public School",
    "Sae College": "SAE College",
    "Sagar Institute Of Research And Technology-Excellence": "Sagar Institute of Research and Technology - Excellence",
    "Saint Agnes Academy": "Saint Agnes Academy",
    "Saint Francis College": "Saint Francis College",
    "Saint Lious University": "Saint Louis University",
    "Saint Louis": "Saint Louis University",
    "Saint Louis University I": "Saint Louis University",
    "Saint Louis University Madrid": "Saint Louis University",
    "Saint Louis University- Madrid": "Saint Louis University",
    "Saint Louis University Mo": "Saint Louis University",
    "Saint Louis University Saint Louis Missouri": "Saint Louis University",
    "Saint Louis University-Madrid": "Saint Louis University",
    "Saint Louis University-Main Campus": "Saint Louis University",
    "Saint Louise University": "Saint Louis University",
    "Saint Louisuniversity": "Saint Louis University",
    "Saint Lousis University": "Saint Louis University",
    "Saint Luis University": "Saint Louis University",
    "Saint.Louis University": "Saint Louis University",
    "Saintlouis": "Saint Louis University",
    "Saintlouis University": "Saint Louis University",
    "St Louis": "Saint Louis University",
    "St Louis University": "Saint Louis University",
    "St. Louis University": "Saint Louis University",
    "SLU": "Saint Louis University",
    "Saint Louis Univeristy": "Saint Louis University",   
    "Saint Louis Univeristy ": "Saint Louis University",  
    "Saint Louis Univeristy.": "Saint Louis University",
    "Saint Louis Univeristy,": "Saint Louis University", 
    "Saint Louis Uinversity": "Saint Louis University",
    "Saint Louis Univercity": "Saint Louis University",
    "Saint Louis Univerisity": "Saint Louis University",
    "Saint Louis University": "Saint Louis University",
    "Saint Louis Univesity": "Saint Louis University",
    "Saint Louis Unniversity": "Saint Louis University",
    "Saint Louis Unveracity": "Saint Louis University",
    "Saint Louis Universty": "Saint Louis University",
    "Saint Louis Universy": "Saint Louis University",
    "Saint Louis": "Saint Louis University",
    "Saint Lious University": "Saint Louis University",
    "Saint Louis": "Saint Louis University",
    "Saint Louis University I": "Saint Louis University",
    "Saint Louis University Madrid": "Saint Louis University",
    "Saint Louis University- Madrid": "Saint Louis University",
    "Saint Louis University Mo": "Saint Louis University",
    "Saint Louis University Saint Louis Missouri": "Saint Louis University",
    "Saint Louis University-Madrid": "Saint Louis University",
    "Saint Louis University-Main Campus": "Saint Louis University",
    "Saint Louise University": "Saint Louis University",
    "Saint Louisuniversity": "Saint Louis University",
    "Saint Lousis University": "Saint Louis University",
    "Saint Luis University": "Saint Louis University",
    "Saint.Louis University": "Saint Louis University",
    "Saintlouis": "Saint Louis University",
    "Saintlouis University": "Saint Louis University",
    "St Louis": "Saint Louis University",
    "St Louis University": "Saint Louis University",
    "St. Louis University": "Saint Louis University",
    "SLU": "Saint Louis University",
    "Slu": "Saint Louis University",
    "Saint Louis Univeristy": "Saint Louis University",
    "Saint Louis Univeristy ": "Saint Louis University",
    "Saint Louis Univeristy.": "Saint Louis University",
    "Saint Louis Univeristy,": "Saint Louis University",
    "Saint Peter'S Minor Seminary Yola": "Saint Peter's Minor Seminary Yola",
    "Sakafiya Islamic Senior High School": "Sakafiya Islamic Senior High School",
    "Saline High School": "Saline High School",
    "Samnabad College": "Samnabad College",
    "Sandip Institute Of Technology And Research Center": "Sandip Institute Of Technology And Research Center",
    "Sandip University": "Sandip University",
    "Sangmeshwar College Solapur": "Sangmeshwar College Solapur",
    "Santa Clara University": "Santa Clara University",
    "Santa Fe College": "Santa Fe College",
    "Sardar Beant Singh State University": "Sardar Beant Singh State University",
    "Sardar Patel Institute Of Technology": "Sardar Patel Institute Of Technology",
    "Sardar Patel University": "Sardar Patel University",
    "Sarghodha University": "Sarghodha University",
    "Sarojini Naidu Vanitha Pharmacy Mahavidyalaya": "Sarojini Naidu Vanitha Pharmacy Mahavidyalaya",
    "Sastra University": "Sastra University",
    "Sathyabama Institute Of Science And Technology": "Sathyabama University",
    "Sathyabama University": "Sathyabama University",
    "Saveetha University": "Saveetha University",
    "Savitribai Phule Pune University": "Savitribai Phule Pune University",
    "Sce Supaul": "SCE Supaul",
    "School For The Gifted": "School For The Gifted",
    "School For The Visual Arts And Humanities": "School For The Visual Arts And Humanities",
    "School Of Nursing": "School of Nursing",
    "School Of Nursing Luth": "School of Nursing",
    "School Of Nursing Ogun State": "School of Nursing",
    "School of Public Health and Social Justice": "School of Public Health and Social Justice",
    "Science College Nanded": "Science College Nanded",
    "SCT College of Engineering": "SCT College of Engineering",
    "SDES": "SDES",
    "Senai Nacional": "Senai Nacional",
    "Seshadri Rao Gudlavalleru Engineering College": "Seshadri Rao Gudlavalleru Engineering College",
    "Seshadrirao Gudlavalleru Engineering College": "Seshadri Rao Gudlavalleru Engineering College",
    "Seventh Day Adventist Higher Secondary School": "Seventh Day Adventist Higher Secondary School",
    "Shah Abdul Latif University Khairpur Mirs": "Shah Abdul Latif University Khairpur Mirs",
    "Shaheed Benazir Bhutto University Nawabshah": "Shaheed Benazir Bhutto University Nawabshah",
    "Shambhunath College Of Education": "Shambhunath College Of Education",
    "Shanmuganathan Engineering College": "Shanmuganathan Engineering College",
    "Shenzhen International Course Center": "Shenzhen International Course Center",
    "Sher Shah Engineering College Sasaram": "Sher Shah Engineering College Sasaram",
    "Shershah Engineering College Sasaram": "Sher Shah Engineering College Sasaram",
    "Shifa Tameer E Millat University": "Shifa Tameer E Millat University",
    "Shivaji": "SHIVAJI",
    "Shree Bhavan'S Bharti Public School": "Shree Bhavan's Bharti Public School",
    "Shri Ram College Of Commerce": "Shri Ram College Of Commerce",
    "Shri Ram College Of Commerce Delhi University": "Shri Ram College Of Commerce",
    "Shri Sant Gajanan Maharaj College Of Engineering Shegaon": "Shri Sant Gajanan Maharaj College Of Engineering Shegaon",
    "Shri Shankaracharya College Of Engineering And Technology": "Shri Shankaracharya College Of Engineering And Technology",
    "Shri Shankarprasad Agnihotri College Of Engineering Wardha": "Shri Shankarprasad Agnihotri College Of Engineering Wardha",
    "Shri Shivaji Science College Amravati": "Shri Shivaji Science College Amravati",
    "Shri Vaishnav Institute Of Information Technology": "Shri Vaishnav Institute Of Information Technology",
    "Sichuan University": "Sichuan University",
    "Siddharth Institute Of Engineering": "Siddharth Institute Of Engineering",
    "Siddhpur Dental College And Hospital": "Siddhpur Dental College And Hospital",
    "Sies": "SIES Graduate School Of Technology",
    "Sies Graduate School Of Technology": "SIES Graduate School Of Technology",
    "Sies Gst Nerul": "SIES Graduate School Of Technology",
    "Sikkim Manipal": "Sikkim Manipal University",
    "Sikkim Manipal University": "Sikkim Manipal University",
    "Sina Hospital": "Sina Hospital",
    "Sinhgad College Of Engineering": "Sinhgad College Of Engineering",
    "Sir Cr Reddy College Of Engineering": "Sir CR Reddy College Of Engineering",
    "Sir Syed University Of Engineering & Technology": "Sir Syed University Of Engineering & Technology",
    "Sjcit": "SJCIT",
    "Sju": "SJU",
    "Smt.Kashibai Navale College Of Engineering": "Smt. Kashibai Navale College Of Engineering",
    "Sn": "SN",
    "South America University": "South America University",
    "Southeast Missouri State University": "Southeast Missouri State University",
    "Southeast University": "Southeast University",
    "Southern New Hampshire University": "Southern New Hampshire University",
    "Southern New Hampshire University Via Kepler College": "Southern New Hampshire University",
    "Southern New Hemisphere University": "Southern New Hampshire University",
    "Southern University": "Southern University",
    "Spd College": "SPD College",
    "Springfield College": "Springfield College",
    "SR Engineering College": "SR Engineering College",
    "Srm": "SRM Institute of Science and Technology",
    "Srm Institute Of Science And Technology, Ramapuram": "SRM Institute of Science and Technology",
    "S.R.M Ist Ramapuram": "SRM Institute of Science and Technology",
    "Srm Institute Of Science And Technology": "SRM Institute of Science and Technology",
    "Srm Institute Of Science And Technology Ramapuram": "SRM Institute of Science and Technology",
    "Srm Ist": "SRM Institute of Science and Technology",
    "Srm University": "SRM Institute of Science and Technology",
    "SRM UNIVERSITY": "SRM Institute of Science and Technology",
    "Srm University Ap": "SRM Institute of Science and Technology",
    "Srm University Ktr": "SRM Institute of Science and Technology",
    "Srm University Ramapuram Chennai": "SRM Institute of Science and Technology",
    "SRM Institute of Science and Technology": "SRM Institute of Science and Technology",
    "SRM Institute Of Science & Technology": "SRM Institute of Science and Technology",
    "Sree Chaitanya College of Engineering": "Sree Chaitanya College of Engineering",
    "Sree Datta Institute of Engineering and Science": "Sree Datta Institute of Engineering and Science",
    "Sree Ratna College of Physiotherapy": "Sree Ratna College of Physiotherapy",
    "Sree Vidyanikethan College Of Pharmacy": "Sree Vidyanikethan College of Pharmacy",
    "Sree Vidyanikethan Engineering College": "Sree Vidyanikethan Engineering College",
    "Sreevidyanikatan Engineering College": "Sree Vidyanikethan Engineering College",
    "Sreenidhi Institute Of Science And Technology": "Sreenidhi Institute Of Science And Technology",
    "Sreenivasa Institute Of Technology And Management Studies": "Sreenivasa Institute Of Technology And Management Studies",
    "Sreyas Institute Of Engineering And Technology": "Sreyas Institute Of Engineering And Technology",
    "Sri Baba Gurudev Degree And Pg College": "Sri Baba Gurudev Degree And PG College",
    "Sri Chaitanya": "Sree Chaitanya College of Engineering",
    "Sri Chandrasekharendra Saraswathi Viswa Mahavidyalaya (Scsvmv)": "Sri Chandrasekharendra Saraswathi Viswa Mahavidyalaya",
    "Sri Guru Gobind Singh College Of Commerce Delhi University": "Sri Guru Gobind Singh College Of Commerce",
    "Sri Guru Tegh Bahadur Khalsa College University Of Delhi": "Sri Guru Tegh Bahadur Khalsa College",
    "Sri Indhu College Of Engineering And Technology": "Sri Indhu College Of Engineering And Technology",
    "Sri Indu": "Sri Indu College Of Engineering And Technology",
    "Sri Indu College Of Engineering And Technology": "Sri Indu College Of Engineering And Technology",
    "Sri Indu Institute Of Engineering And Technology": "Sri Indu Institute Of Engineering And Technology",
    "Sri Indu Institute Of Management And Technology": "Sri Indu Institute Of Management And Technology",
    "Sri Krishna Arts And Science College": "Sri Krishna Arts And Science College",
    "Sri Krishna Institute Of Technology": "Sri Krishna Institute Of Technology",
    "Sri Krishna Institution Of Technology": "Sri Krishna Institute Of Technology",
    "Sri Ramakrishna Engineering College": "Sri Ramakrishna Engineering College",
    "Sri Sai College Of Dental Surgery": "Sri Sai College Of Dental Surgery",
    "Sri Sairam Engineering College": "Sri Sairam Engineering College",
    "Sri Satya Sai University Of Technology And Medical Science": "Sri Satya Sai University Of Technology And Medical Science",
    "Sri Vasavi Engineering College": "Sri Vasavi Engineering College",
    "Sri Vasavi Institute Of Engineering And Technology": "Sri Vasavi Institute Of Engineering And Technology",
    "Sri Venkateswara College Of Engineering": "Sri Venkateswara College Of Engineering",
    "Sri Venkateswara College Of Engineering And Technology": "Sri Venkateswara College Of Engineering And Technology",
    "Sri Yn College": "Sri Yn College",
    "Srkr Engineering College": "SRKR Engineering College",
    "Sscbs": "SSCBS",
    "Ssit": "SSIT",
    "St Aloysius Pu College Mangalore": "St. Aloysius PU College Mangalore",
    "St Andrew'S Senior High School": "St. Andrew's Senior High School",
    "St Augustine College": "St. Augustine College",
    "St Joseph'S College Of Engineering": "St. Joseph's College Of Engineering",
    "St Mary'S Engineering College": "St. Mary’s Engineering College",
    "St Peters Institute Of Higher Education And Research": "St. Peters Institute Of Higher Education And Research",
    "St Thomas College": "St. Thomas College",
    "St Thomas School": "St. Thomas School",
    "St. Ann'S College Of Engineering And Technology": "St. Ann's College Of Engineering And Technology",
    "St. John'S College Palayamkottai.": "St. John's College Palayamkottai",
    "St. John'S University": "St. John's University",
    "St. Joseph Engineering College": "St. Joseph Engineering College",
    "St. Joseph'S Academy": "St. Joseph's Academy",
    "St. Joseph'S Degree And Pg College": "St. Joseph's Degree And PG College",
    "St. Joseph'S Degree&Pg College": "St. Joseph's Degree And PG College",
    "St.Joseph'S Degree And Pg College": "St. Joseph's Degree And PG College",
    "St.Joseph'S Institute Of Technology": "St. Joseph's Institute Of Technology",
    "St.Joseph'S University": "St. Joseph's University",
    "St. Karol School Of Nursing": "St. Karol School Of Nursing",
    "St. Louis Senior High School": "St. Louis Senior High School",
    "St. Mary'S Dominican High School": "St. Mary’s Dominican High School",
    "St. Marys Group Of Institutions Guntur": "St. Mary’s Group Of Institutions Guntur",
    "St. Paul Technical School": "St. Paul Technical School",
    "St. Teresa'S College": "St. Teresa's College",
    "St. Thomas Aquinas Senior High School": "St. Thomas Aquinas Senior High School",
    "St. Thomas School": "St. Thomas School",
    "St. Vincent High And Technical School": "St. Vincent High And Technical School",
    "St. Xaviers College": "St. Xaviers College",
    "St.Francis College For Women": "St. Francis College For Women",
    "St.Francis Colloege For Women": "St. Francis College For Women",
    "St.Martins Engineering College": "St. Martins Engineering College",
    "Starlet High School": "Starlet High School",
    "State Institute Of Engineering & Technology Nilokheri": "State Institute Of Engineering & Technology Nilokheri",
    "Stony Brook": "Stony Brook University",
    "Suez Canal Uni": "Suez Canal University",
    "Suez Canal University": "Suez Canal University",
    "Sukkur Iba University": "Sukkur IBA University",
    "Sule Lamido University": "Sule Lamido University",
    "Sultan Moulay Slimane University": "Sultan Moulay Slimane University",
    "Superior": "Superior University",
    "Superior Group Of College": "Superior University",
    "Superior University": "Superior University",
    "Sushila Devi Bansal College Of Technology Indore": "Sushila Devi Bansal College Of Technology",
    "Sv College Of Engineering For Women": "SV College Of Engineering For Women",
    "Svit Vasad": "SVIT Vasad",
    "Svnit": "SVNIT",
    "Svsps": "SVSPS",
    "Swami Ramanand Teerth Marathwada University Nanded": "Swami Ramanand Teerth Marathwada University",
    "Swami Vivekanand Collage": "Swami Vivekanand College",
    "Swarandhra": "Swarnandhra Institute Of Engineering & Technology",
    "Swarnandhra Institute Of Engg & Technology": "Swarnandhra Institute Of Engineering & Technology",
    "Swarnandhra Institute Of Engineering And Technology": "Swarnandhra Institute Of Engineering & Technology",
    "Symbiosis Center For Distance Learning (Scdl)": "Symbiosis Center For Distance Learning",
    "Symbiosis Institute Of Technoogy": "Symbiosis Institute Of Technology",
    "Symbiosis International Deemed University": "Symbiosis International University",
    "Symbiosis International University": "Symbiosis International University",
    "Symbiosis Statistical Institute": "Symbiosis Statistical Institute",
    "Symboisis": "Symbiosis International University",
    "S P D College": "SPD College",
    "S.J.C.I.T": "SJCIT",
    "S.R.M Ist Ramapuram": "SRM Institute of Science and Technology",
    "Srm Institute Of Science And Technology": "SRM Institute of Science and Technology",
    "Srm Institute Of Science And Technology Ramapuram": "SRM Institute of Science and Technology",
    "Srm Ist": "SRM Institute of Science and Technology",
    "Srm University": "SRM Institute of Science and Technology",
    "SRM UNIVERSITY": "SRM Institute of Science and Technology",
    "Sr Engineering College": "SR Engineering College",
    "Sr University": "SR University",
    "St Joseph Engineering College": "St. Joseph Engineering College",
    "St Joseph’S College Of Engineering": "St. Joseph’s College of Engineering",
    "St Joseph'S Academy": "St. Joseph's Academy",
    "Srcc": "Shri Ram College of Commerce",
    "Sree Chaitanya College Of Engineering": "Sree Chaitanya College of Engineering",
    "Sree Datta Institute Of Engineering And Science": "Sree Datta Institute of Engineering and Science",
    "Sree Dattha Institute Of Engineering And Science": "Sree Datta Institute of Engineering and Science",
    "Sree Ratna College Of Physiotherapy": "Sree Ratna College of Physiotherapy",
    "Sree Vidyanikehan Engineering College": "Sree Vidyanikethan Engineering College",
    "Seshadri Ra": "Seshadri Rao Gudlavalleru Engineering College",
    "Seshadri Rao Gudlavalleru Engineering College": "Seshadri Rao Gudlavalleru Engineering College",
    "School Of Post Graduate Studies Nigerian Defense Academy": "Nigerian Defence Academy, School of Postgraduate Studies",
    "School Of Public Health And Social Justice": "School of Public Health and Social Justice",
    "Science College Nanded": "Science College Nanded",
    "Sct College Of Engineering": "SCT College of Engineering",
    "Sdes": "SDES",
    "Senai Nacional": "Senai Nacional",
    "Sershah Engineering College Sasaram": "Sher Shah Engineering College Sasaram",
    "T.N.B. College": "T.N.B. College",
    "Tai Solarin University Of Education": "Tai Solarin University Of Education",
    "Takoradi Technical University": "Takoradi Technical University",
    "Tanzania Institute Of Accountancy": "Tanzania Institute Of Accountancy",
    "Tech4Dev": "Tech4Dev",
    "Technical University Of Kenya": "Technical University Of Kenya",
    "Techno International Batanagar": "Techno International Batanagar",
    "Techno Main Saltlake": "Techno Main Saltlake",
    "Technocrats Institute Of Technology Excellence": "Technocrats Institute Of Technology Excellence",
    "Ted Ankara College": "Ted Ankara College",
    "Teerthanker Mahaveer University": "Teerthanker Mahaveer University",
    "Tekedia Institute": "Tekedia Institute",
    "Ternopil National Medical University": "Ternopil National Medical University",
    "Teshie Presbyterian Senior High School": "Teshie Presbyterian Senior High School",
    "Texas A&M University": "Texas A&M University",
    "Thakur College Of Engineering And Technology": "Thakur College Of Engineering And Technology",
    "Thamar University": "Thamar University",
    "The Apostolic Secondary Grammar School Ugep": "The Apostolic Secondary Grammar School Ugep",
    "The Bishop'S School": "The Bishop's School",
    "The British University In Egypt": "The British University In Egypt",
    "The Institute Of Chartered Accountants Of Pakistan": "The Institute Of Chartered Accountants Of Pakistan",
    "The Islamia University Of Bhwalpur": "The Islamia University Of Bahawalpur",
    "The Maharaja Sayajirao University Of Baroda": "The Maharaja Sayajirao University Of Baroda",
    "The Millennium Universal College": "The Millennium Universal College",
    "The National University Of Science And Technology Nust": "The National University Of Science And Technology (NUST)",
    "The Ngp School": "The NGP School",
    "The Northcap University": "The NorthCap University",
    "The Oke Ogun Polytechnic": "The Oke Ogun Polytechnic",
    "The Oke Ogun Polytechnic Saki Oyo State": "The Oke Ogun Polytechnic",
    "The Open University": "The Open University",
    "The Polytechnic Ibadan Oyo State": "The Polytechnic Ibadan",
    "The Study Hall College": "The Study Hall College",
    "The University Of Arizona": "University of Arizona",
    "The University Of Lahore": "University of Lahore",
    "The University Of Queensland": "University of Queensland",
    "The University Of Texas At Austin": "University of Texas at Austin",
    "The University Of Texas At Dallas": "University of Texas at Dallas",
    "Thiagarajar College Of Engineering": "Thiagarajar College Of Engineering",
    "Thnwy@ Bn Syn Lt'Hyly@": "Unknown",
    "Thomas Edison State University": "Thomas Edison State University",
    "Thuong Mai University": "Thuong Mai University",
    "Tjit": "TJIT",
    "Tjohn Institute Of Technology": "TJohn Institute Of Technology",
    "Tkr College Of Engineering And Technology": "TKR College Of Engineering And Technology",
    "Touro College Of Pharmacy": "Touro College Of Pharmacy",
    "Tribhuvan University": "Tribhuvan University",
    "Trinity Christian School": "Trinity Christian School",
    "Trinity College Of Engineering And Research": "Trinity College Of Engineering And Research",
    "Truba Institute Of Science And Technology": "Truba Institute Of Science And Technology",
    "Truong Dai Hoc Kinh Te Quoc Dan": "Truong Dai Hoc Kinh Te Quoc Dan",
    "Tss": "TSS",
    "Tufanganj N.N.M. High School": "Tufanganj N.N.M. High School",
    "Tula'S Institute Dehradun": "Tula's Institute Dehradun",
    "Tuljaram Chaturchand College Of Art And Science And Commerce": "Tuljaram Chaturchand College Of Art And Science And Commerce",
    "Uet Taxila": "UET Taxila",
    "Uganda Christian University": "Uganda Christian University",
    "Uin Walisongo": "UIN Walisongo",
    "Uit": "UIT",
    "Uit Of Douala": "UIT Douala",
    "Ujjain Engineering College": "Ujjain Engineering College",
    "Universite Felix Houphouet Boigny Cocody Abidjan": "Université Félix Houphouët-Boigny, Abidjan",
    "University College Cork": "University College Cork",
    "University College Hospital. Ibadan.": "University College Hospital, Ibadan",
    "University College Of Engineering Kakinada (Autonomous) Jawaharlal Nehru Technological University Kakinada": "University College of Engineering, JNTU Kakinada",
    "University College Of Technology Osmania University": "University College of Technology, Osmania University",
    "University For Development Studies": "University for Development Studies",
    "University Of Abuja": "University of Abuja",
    "University Of Agriculture Faisalabad": "University of Agriculture Faisalabad",
    "University Of Agriculture Faislabad Pakistan": "University of Agriculture Faisalabad",
    "University Of Applied Sciences Aachen": "University of Applied Sciences Aachen",
    "University Of Applied Sciences Soest": "University of Applied Sciences Soest",
    "University Of Arizona": "University of Arizona",
    "University Of Bamenda": "University of Bamenda",
    "University Of Benin": "University of Benin, Edo State",
    "University Of Benin Edo State": "University of Benin, Edo State",
    "University Of Birmingham": "University of Birmingham",
    "University Of Bolton": "University of Bolton",
    "University Of Botswana": "University of Botswana",
    "University Of Bridgeport": "University of Bridgeport",
    "University Of Calabar": "University of Calabar",
    "University Of Calabar Calabar": "University of Calabar",
    "University Of Calabar Crs Nigeria": "University of Calabar",
    "University Of Cape Coast": "University of Cape Coast",
    "University Of Capecoast": "University of Cape Coast",
    "University Of Central Missouri": "University of Central Missouri",
    "University Of Chester": "University of Chester",
    "University Of Colombo": "University of Colombo",
    "University Of Connecticut": "University of Connecticut",
    "University Of Dar Es Salaam": "University of Dar es Salaam",
    "University Of Dayton": "University of Dayton",
    "University Of Debrecen": "University of Debrecen",
    "University Of Delhi": "University of Delhi",
    "University Of Doha For Science And Technology": "University of Doha for Science and Technology",
    "University Of East London": "University of East London",
    "University Of Economics Hochiminh City": "University of Economics Ho Chi Minh City",
    "University Of Education": "University of Education",
    "University Of Education Lahore": "University of Education",
    "University Of Education Winneba": "University of Education",
    "University Of Energy And Natural Resources": "University of Energy and Natural Resources",
    "University Of Engineering And Technology": "University of Engineering and Technology",
    "University Of Engineering And Technology Lahore": "University of Engineering and Technology, Lahore",
    "University Of Europe": "University of Europe",
    "University Of Exeter": "University of Exeter",
    "University Of Ghana": "University of Ghana",
    "University Of Ghana Medical School": "University of Ghana Medical School",
    "University Of Gonder": "University of Gondar",
    "University Of Gujrat": "University of Gujrat",
    "University Of Hawaii At Manoa": "University of Hawaii at Manoa",
    "University Of Health And Allied Sciences": "University of Health and Allied Sciences",
    "University Of Hull": "University of Hull",
    "University Of Ibadan": "University of Ibadan",
    "University Of Ibadan Ibadan Nigeria.": "University of Ibadan",
    "University Of Illinois At Springfield": "University of Illinois Springfield",
    "University Of Illinois Springfield": "University of Illinois Springfield",
    "University Of Ilorin": "University of Ilorin",
    "University Of Ilorin Ilorin Kwara State": "University of Ilorin",
    "University Of Ilorin Kwara-Nigeria": "University of Ilorin",
    "University Of Ilorin Teaching Hospital": "University of Ilorin Teaching Hospital",
    "University Of Johannesburg": "University of Johannesburg",
    "University Of Jos": "University of Jos",
    "University Of Kashmir": "University of Kashmir",
    "University Of Kerala": "University of Kerala",
    "University Of Khartoum": "University of Khartoum",
    "University Of Lagos": "University of Lagos",
    "University Of Lagos Nigeria": "University of Lagos",
    "University Of Lahore": "University of Lahore",
    "University Of Lucknow": "University of Lucknow",
    "University Of Maiduguri": "University of Maiduguri",
    "University Of Maiduguri Borno State": "University of Maiduguri",
    "University Of Management And Technology": "University of Management and Technology",
    "University Of Management And Technology Lahore Pakistan": "University of Management and Technology",
    "University Of Medecine": "University of Medical Sciences",
    "University Of Medical Science": "University of Medical Sciences",
    "University Of Medical Sciences": "University of Medical Sciences",
    "University Of Mines And Technology": "University of Mines and Technology",
    "University Of Mines And Technology Tarkwa": "University of Mines and Technology",
    "University Of Missouri Saint Louis": "University of Missouri–Saint Louis",
    "University Of Mumbai": "University of Mumbai",
    "University Of Nairobi": "University of Nairobi",
    "University Of Namibia": "University of Namibia",
    "University Of Nigeria": "University of Nigeria",
    "University Of Nigeria Enugu Campus": "University of Nigeria",
    "University Of Nigeria Nsukka": "University of Nigeria",
    "University Of Okara": "University of Okara",
    "University Of Petroleum And Energy Studies": "University of Petroleum and Energy Studies",
    "University Of Phoenix": "University of Phoenix",
    "University Of Poonch Rawlakot": "University of Poonch Rawlakot",
    "University Of Port Harcourt": "University of Port Harcourt",
    "University Of Port Harcourt Business School": "University of Port Harcourt",
    "University Of Port Harcourt Nigeria": "University of Port Harcourt",
    "University Of Port Harcourt Rivers State": "University of Port Harcourt",
    "University Of Portharcourt": "University of Port Harcourt",
    "University Of Port-Harcourt": "University of Port Harcourt",
    "University Of Professional Studies Accra": "University of Professional Studies Accra",
    "University Of Punjab": "University of Punjab",
    "University Of Rajshahi": "University of Rajshahi",
    "University Of Rwanda": "University of Rwanda",
    "University Of Santo Tomas Senior High School": "University of Santo Tomas Senior High School",
    "University Of Sargodha": "University of Sargodha",
    "University Of Sindh Jamshoro": "University of Sindh Jamshoro",
    "University Of Southern California": "University of Southern California",
    "University Of Technology": "University of Technology",
    "University Of Tehran": "University of Tehran",
    "University Of Texas At Austin": "University of Texas at Austin",
    "University Of Texas At Dallas": "University of Texas at Dallas",
    "University Of Texas Rio Grande Valley": "University of Texas Rio Grande Valley",
    "University Of The Central Punjab": "University of the Central Punjab",
    "University Of The Gambia": "University of the Gambia",
    "University Of The People": "University of the People",
    "University Of The Punjab": "University of the Punjab",
    "University Of The Punjab Lahore": "University of the Punjab",
    "University Of Toronto Mississauga": "University of Toronto Mississauga",
    "University Of Twente": "University of Twente",
    "University Of Uyo": "University of Uyo",
    "University Of Uyo Akwaibom State": "University of Uyo",
    "University Of Uyo Nigeria": "University of Uyo",
    "University Of Wah": "University of Wah",
    "University Of Wisconsin-Madison": "University of Wisconsin–Madison",
    "University Of Wollongong Australia": "University of Wollongong",
    "University Of Zimbabwe": "University of Zimbabwe",
    "Uop Pakistan": "University of the People",
    "Uopeople": "University of the People",
    "Upgrad": "UPGRAD",
    "Upsa": "UPSA",
    "Uptu": "UPTU",
    "Uskudar University": "Uskudar University",
    "Usman Danfodiyo University Nigeria": "Usman Danfodiyo University",
    "Usmanu Danfodiyo University": "Usman Danfodiyo University",
    "Usman Institute Of Technology": "Usman Institute of Technology",
    "Utem": "UTEM",
    "Utiva": "Utiva",
    "V R Siddhartha Engineering College": "V R Siddhartha Engineering College",
    "V. K. R V. N. B & A. G. K College Of Engineering": "V. K. R V. N. B & A. G. K College of Engineering",
    "Vaagdevi Engineering College": "Vaagdevi Engineering College",
    "Vaagdevi Engineering College": "Vaagdevi Engineering College",
    "Valley View University": "Valley View University",
    "Vardhaman College Of Engineering": "Vardhaman College of Engineering",
    "Vasireddy Venkatadri Institute Of Technology": "Vasireddy Venkatadri Institute of Technology",
    "Veer Bahadur Singh Purvanchal University": "Veer Bahadur Singh Purvanchal University",
    "Veer Narmad South Gujarat University": "Veer Narmad South Gujarat University",
    "Veermata Jijabai Technological Institue ( Vjti)": "Veermata Jijabai Technological Institute (VJTI)",
    "Velagapudi": "Velagapudi Ramakrishna Siddhartha Engineering College",
    "Velgapudi Ramakrishna Siddhartha Engineering College": "Velagapudi Ramakrishna Siddhartha Engineering College",
    "Vellore Institute Of Technology": "Vellore Institute of Technology",
    "Vellore Institute Of Technology - Andhra Pradesh": "Vellore Institute of Technology",
    "Vellore Institute Of Technology Amaravati": "Vellore Institute of Technology",
    "Vellore Institute Of Technology Bhopal": "Vellore Institute of Technology",
    "Vellore Institute Of Technology Vellore": "Vellore Institute of Technology",
    "Vels Institute Of Science And Technology": "Vels Institute of Science and Technology",
    "Veltech Dr Rr & Dr Sr Institute Of Science And Technology": "Veltech Dr RR & Dr SR Institute of Science and Technology",
    "Vemana Institute Of Technology": "Vemana Institute of Technology",
    "Vetka High School": "Vetka High School",
    "Vi Featuring Institution": "Vi Featuring Institution",
    "Vibgyor": "Vibgyor High",
    "Vibgyor High": "Vibgyor High",
    "Vibgyor High Nibm": "Vibgyor High",
    "Vicon School": "Vicon School",
    "Victoria Institution College": "Victoria Institution College",
    "Victoria University": "Victoria University",
    "Vidhya Prathisthan College Of Engineering Baramati": "Vidhya Prathisthan College of Engineering Baramati",
    "Vidya Jyothi Institute Of Technology": "Vidya Jyothi Institute of Technology",
    "Vidya Pratishthan Kamalnayan Bajaj Institute Of Engineering And Technology Baramati": "Vidya Pratishthan Kamalnayan Bajaj Institute of Engineering and Technology",
    "Vidya Pratishthans Kamal Nayan Bajaj Institute Of Engineering And Technology": "Vidya Pratishthan Kamalnayan Bajaj Institute of Engineering and Technology",
    "Vidya Valley International": "Vidya Valley International",
    "Vidyalankar Institute Of Technology": "Vidyalankar Institute of Technology",
    "Vignan": "Vignan",
    "Vignan Degree And Pg College": "Vignan Degree and PG College",
    "Vignan Institute Of Technology And Science": "Vignan Institute of Technology and Science",
    "Vignan Nirula Institute Of Technology": "Vignan Nirula Institute of Technology",
    "Vignan University": "Vignan University",
    "Vignan University For Science Research And Technology": "Vignan University",
    "Vignana Bharathi Institute Of Technology": "Vignana Bharathi Institute of Technology",
    "Vignana Jyothi Institute Of Arts And Sciences": "Vignana Jyothi Institute of Arts and Sciences",
    "Vignan'S Foundation For Science Research And Technology": "Vignan's Foundation for Science Research and Technology",
    "Vignans Institute Of Information Technology": "Vignan's Institute of Information Technology",
    "Vignan'S Institute Of Information Technology": "Vignan's Institute of Information Technology",
    "Vignan'S Lara Institute Of Technology & Science": "Vignan's Lara Institute of Technology and Science",
    "Vignan'S Lara Institute Of Technology And Science": "Vignan's Lara Institute of Technology and Science",
    "Vignan'S Lara Institute Of Technology And Sciences": "Vignan's Lara Institute of Technology and Science",
    "Vignan'S Nirula Institute Of Technology And Science For Women.": "Vignan Nirula Institute of Technology for Women",
    "Virtual University": "Virtual University of Pakistan",
    "Virtual University Of Pakistan": "Virtual University of Pakistan",
    "Vishay Bharti Mahavidhaylay Amravati": "Vishay Bharti Mahavidhaylay Amravati",
    "Vishnu Institute Of Technology": "Vishnu Institute of Technology",
    "Vishwakarma Institute Of Technology Pune": "Vishwakarma Institute of Technology Pune",
    "Vishwaniketan'S Institute Of Management Entrepreneurship And Engineering Technology": "Vishwaniketan's Institute of Management, Entrepreneurship and Engineering Technology",
    "Visvesvaraya Technological University": "Visvesvaraya Technological University",
    "Visveswaraya Technical University": "Visvesvaraya Technological University",
    "Vit Ap University": "VIT-AP University",
    "Vit Bhopal University": "VIT Bhopal University",
    "Vit Chennai": "VIT Chennai",
    "Vit Icer": "VIT ICER",
    "Vit University": "VIT University",
    "Vit University Vellore": "VIT University",
    "Vit-Ap University": "VIT-AP University",
    "Vivekananda College Af Arts Science And Commerce (Autonomous)": "Vivekananda College of Arts, Science and Commerce",
    "Vnit": "VNIT",
    "Vnr Vignana Jyothi Institute Of Engineering And Technology": "VNR Vignana Jyothi Institute of Engineering and Technology",
    "Vnr Vignana Jyothi Institute Of Enginnering": "VNR Vignana Jyothi Institute of Engineering and Technology",
    "Vnrvjiet": "VNR Vignana Jyothi Institute of Engineering and Technology",
    "Vocational High School 1 Tangerang City": "Vocational High School 1 Tangerang City",
    "Vpkbiet": "VPKBIET Baramati",
    "Vpkbiet Baramati": "VPKBIET Baramati",
    "Vr Siddhartha Engineering College": "V R Siddhartha Engineering College",
    "Vtu": "VTU",
    "Vvit": "VVIT",
    "Vyshanavi Degree Coliege": "Vyshanavi Degree College",
    "Wa Nursing Training College": "WA Nursing Training College",
    "Wah Engineering College": "Wah Engineering College",
    "Waubonsie Valley High School": "Waubonsie Valley High School",
    "Webster": "Webster University",
    "Webster University": "Webster University",
    "Wesley Girls High School": "Wesley Girls High School",
    "West End University College": "West End University College",
    "Western Governors University": "Western Governors University",
    "Whitfield School": "Whitfield School",
    "Wilby High School": "Wilby High School",
    "Wildlife Clubs Of Kenya": "Wildlife Clubs of Kenya",
    "Wisconsin International University College": "Wisconsin International University College",
    "Wollo University": "Wollo University",
    "Xime Bangalore": "XIME Bangalore",
    "Yaba College Of Technology": "Yaba College of Technology",
    "Yabatech": "Yaba College of Technology",
    "Yan Zhou Shi Shi Yan Wai Yu Xue Xiao": "Yan Zhou Shi Shi Yan Wai Yu Xue Xiao",
    "Yashwant Rao Chavan University Nashik": "Yashwantrao Chavan University, Nashik",
    "Yes Christian Academy": "Yes Christian Academy",
    "Yeshwantrao Chavan College Of Engineering": "Yeshwantrao Chavan College of Engineering",
    "Yishun Junior College": "Yishun Junior College",
    "Yokohama Ichiritsu Kanazawa Koko": "Yokohama Ichiritsu Kanazawa Koko",
    "Ysr Engineering College": "YSR Engineering College",
    "Zagazig University": "Zagazig University",
    "Zambia University College Of Technology": "Zambia University College of Technology",
    "(C) 2023 Tomtom (C) Openstreetmap Sri Venkateswara College Of Engineering": "Sri Venkateswara College of Engineering",
    "(c) 2023 Tomtom (c) Openstreetmap Sri Venkateswara College Of Engineering": "Sri Venkateswara College of Engineering",
    "© 2023 TomTom  © OpenStreetMap Sri Venkateswara College Of Engineering": "Sri Venkateswara College of Engineering"
}

INVALID_INSTITUTES = {
    "Na", "NA", "Nil", "None", "Unknown", "Not Applicable", "Null", "Applying", 
    "Awaiting Graduate Admissions Into St. Louis", "Employment", "Excelarate",
    "Excelr", "Asdads", "Asdf", "Job", "Test", "Other", "Oth", "Student", 
    "Fresher", "I Am Major", "Aa", "Abc", "Cdwcnwj" , "Englis", "Habiba",
    "Habibrehman529@Gmail.Com", "Graphics Designing", "Helpline", "Hindustan",
     "Homeschooled", "Hsc", "I", "I Am Currently In My Gap Year", "Ill", "Ind","Irs", "Qq" 
}

def clean_institution(name):
    if pd.isna(name):
        return pd.NA
    name = str(name).strip()
    name = unidecode(name) 
    name = re.sub(r'\s+', ' ', name)  

    if name in INVALID_INSTITUTES:
        return pd.NA
    if name in INSTITUTE_CORRECTIONS:
        return INSTITUTE_CORRECTIONS[name]

    return name.title()
    
df['Institution Name'] = df['Institution Name'].apply(clean_institution)

print(df['Institution Name'].nunique(), "unique institutions after cleaning")
print(df['Institution Name'].value_counts().head(20))

1446 unique institutions after cleaning
Institution Name
Saint Louis University                                4517
Illinois Institute of Technology                       176
Webster University                                      73
SRM Institute of Science and Technology                 60
Kwame Nkrumah University of Science and Technology      41
Jawaharlal Nehru Technological University               40
University of Port Harcourt                             31
Vellore Institute of Technology                         30
Federal University Of Technology                        28
University of Ibadan                                    27
Lotus Valley International School                       27
Vishnu Institute of Technology                          25
University of Ilorin                                    24
Chandigarh University                                   24
University of Nigeria                                   23
University of Benin, Edo State                          22

In [647]:
# Fill missing First Names with 'Not Specified'
df['First Name'] = df['First Name'].fillna('Not Specified')

# Define the strong grouping columns
group_cols = ['First Name', 'Date of Birth', 'Country']

# Impute by filling nulls with the first non-null value from the person's group
df['Institution Name'] = (
    df['Institution Name']
    .fillna(df.groupby(group_cols)['Institution Name'].transform('first'))
)
df['Current/Intended Major'] = (
    df['Current/Intended Major']
    .fillna(df.groupby(group_cols)['Current/Intended Major'].transform('first'))
)
# After the smart fill, handle any remaining nulls for people who never provided the info
df['Institution Name'] = df['Institution Name'].fillna('Not Specified')
df['Current/Intended Major'] = df['Current/Intended Major'].fillna('Not Specified')

In [649]:
# Ensure datetime conversion
df['Opportunity End Date'] = pd.to_datetime(df['Opportunity End Date'], errors='coerce')
df['Opportunity Start Date'] = pd.to_datetime(df['Opportunity Start Date'], errors='coerce')

# Calculate average duration for "Competition" only
competition_duration = (df.loc[
    (df['Opportunity Category'] == 'Competition') & df['Opportunity Start Date'].notna(),
    'Opportunity End Date'
] - df.loc[
    (df['Opportunity Category'] == 'Competition') & df['Opportunity Start Date'].notna(),
    'Opportunity Start Date'
]).dt.days.mean()

# Fill nulls only for "Competition"
mask = (df['Opportunity Category'] == 'Competition') & df['Opportunity Start Date'].isna()
df.loc[mask, 'Opportunity Start Date'] = df.loc[mask, 'Opportunity End Date'] - pd.to_timedelta(competition_duration, unit='D')


In [651]:
# Check the data after cleaning
print(df.info())
print(df.isna().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8558 entries, 0 to 8557
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Learner SignUp DateTime  8558 non-null   datetime64[ns]
 1   Opportunity Id           8558 non-null   object        
 2   Opportunity Name         8558 non-null   object        
 3   Opportunity Category     8558 non-null   object        
 4   Opportunity End Date     8558 non-null   datetime64[ns]
 5   First Name               8558 non-null   object        
 6   Date of Birth            8558 non-null   object        
 7   Gender                   8558 non-null   object        
 8   Country                  8558 non-null   object        
 9   Institution Name         8558 non-null   object        
 10  Current/Intended Major   8558 non-null   object        
 11  Entry created at         8558 non-null   datetime64[ns]
 12  Status Description       8558 non-

In [79]:
df.to_csv("cleaned_data.csv", index=False)
print("Cleaned data saved as cleaned_data.csv")

Cleaned data saved as cleaned_data.csv
